In [63]:
import psycopg2
import re
import pandas as pd
import numpy as np
import math
import statistics

#for eICU
conn = psycopg2.connect(host = "address",database = "databasename"
                        , user = "username", password = "")
#how long would you like to take in minutes(24hr = 1440min)
strsql_timeoffst = '1440'
cur = conn.cursor()


In [36]:
#get AKI patient's ID and diagnosisoffset
cur.execute("SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid")
akiDiagdf = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','diagnosisoffset'])


#net fluid balance
cur = conn.cursor()
cur.execute("SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.nettotal FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY IO.patientunitstayid,IO.intakeoutputoffset")
IO_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','intakeoutputoffset','nettotal'])
IO_res = IO_res.groupby(['patientunitstayid','intakeoutputoffset']).agg({'nettotal':np.min}).reset_index()
IO_res = IO_res.groupby(['patientunitstayid']).agg({'nettotal':np.sum}).reset_index()  
nettotal = IO_res
nettotal.columns = ['patientunitstayid','nettotal']
nettotal.set_index('patientunitstayid',inplace = True)
nettotal = nettotal.fillna(1000000)
nettotal = pd.DataFrame(nettotal,dtype = np.float)
for i in nettotal[(nettotal['nettotal'] != 1000000) & (nettotal['nettotal'] > 3000)].index.values:
    nettotal.loc[i]['nettotal'] = 1000000
aki_nettotal = nettotal

print("nettotal Missing Rate:",1 - nettotal[nettotal['nettotal']!=1000000]['nettotal'].count()/19615)
print(nettotal)


nettotal Missing Rate: 0.3391282182003569
                     nettotal
patientunitstayid            
141266                  63.22
141304                1253.00
141314                 150.00
141448                   0.00
141470                   0.00
...                       ...
3353194               -468.00
3353197            1000000.00
3353199                 17.44
3353251                 60.13
3353254                500.00

[14399 rows x 1 columns]


In [97]:
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
potassium_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
potassium_res = potassium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
potassium_size = potassium_res.groupby("patientunitstayid").size()
print("potassium max columns:" + str(max(potassium_size)) + "\n" + "potassium avg columns:" + str(sum(potassium_size)/len(potassium_size)) + "\n" + "potassium missing rate:" + str((19615-len(potassium_size))/19615))
#potassium_res.set_index(['patientunitstayid','labresultoffset']).index.is_unique

length_potassium = int(statistics.median(potassium_size))

#potassium turn continuous data from vertical to horizontal
potassium_res['cumCount'] = potassium_res.groupby(['patientunitstayid']).cumcount()
potassium_1 = potassium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
potassium_1 = potassium_1.iloc[:,:length_potassium]

#potassium let format right
potassium_1 = pd.DataFrame(potassium_1)
potassium_1 = potassium_1.fillna(1000000)
potassium_1 = pd.DataFrame(potassium_1,dtype = np.float)
column = []
i = 0
while i < length_potassium:
    column.append("potassium_" + "%s"%str(i))
    i += 1
potassium_1.columns = column
potassium_1 = pd.DataFrame(potassium_1.index)
potassium_1.set_index('patientunitstayid',inplace = True)
potassium_1


potassium max columns:38
potassium avg columns:1.7984875983061102
potassium missing rate:0.15727759367830743


""
patientunitstayid
141266
141304
141314
141448
141470
...
3353190
3353194
3353197


In [101]:
#potassium
i = 0
offset = 240
NextOffset = 0
potassium_dflist = []
potassium_tdflist = []
i = int(int(strsql_timeoffst)/offset)
while i > 0:
    PreOffset = offset*(i - 1)
    NextOffset = offset*i
    sql = "SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY L.patientunitstayid,L.labresultoffset"
    cur.execute(sql)
    potassium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset_'+str(i),'potassium_'+str(i)])
    potassium_tdflist.append(potassium_res.groupby(['patientunitstayid']).agg({'labresultoffset_'+str(i):np.sum}).reset_index())
    potassium_res = potassium_res.groupby(['patientunitstayid']).agg({'potassium_'+str(i):np.sum}).reset_index() 
    potassium_dflist.append(potassium_res)
    i -= 1


potassium = pd.DataFrame()
potassium = potassium_1
for data in potassium_dflist[0:]:
    potassium = pd.merge(potassium,data,on = 'patientunitstayid',how = 'left')
potassium = potassium.set_index('patientunitstayid')
potassium = potassium.fillna(1000000)

potassium[(potassium['potassium_6'] == 1000000) & (potassium['potassium_5'] == 1000000) & (potassium['potassium_4'] == 1000000) & (potassium['potassium_3'] == 1000000) & (potassium['potassium_2'] == 1000000) & (potassium['potassium_1'] == 1000000)].count


<bound method DataFrame.count of Empty DataFrame
Columns: [potassium_6, potassium_5, potassium_4, potassium_3, potassium_2, potassium_1]
Index: []>

In [37]:
#potassium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
potassium_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
potassium_res = potassium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
potassium_size = potassium_res.groupby("patientunitstayid").size()
print("potassium max columns:" + str(max(potassium_size)) + "\n" + "potassium avg columns:" + str(sum(potassium_size)/len(potassium_size)) + "\n" + "potassium missing rate:" + str((19615-len(potassium_size))/19615))
#potassium_res.set_index(['patientunitstayid','labresultoffset']).index.is_unique

length_potassium = int(statistics.median(potassium_size))

#potassium turn continuous data from vertical to horizontal
potassium_res['cumCount'] = potassium_res.groupby(['patientunitstayid']).cumcount()
potassium = potassium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
potassium = potassium.iloc[:,:length_potassium]

#potassium let format right
potassium = pd.DataFrame(potassium)
potassium = potassium.fillna(1000000)
potassium = pd.DataFrame(potassium,dtype = np.float)
column = []
i = 0
while i < length_potassium:
    column.append("potassium_" + "%s"%str(i))
    i += 1
potassium.columns = column

#caculate delta

#store the measure offset
potassium_time = potassium_res
#potassium_time turn continuous data from vertical to horizontal
potassium_time['cumCount'] = potassium_time.groupby(['patientunitstayid']).cumcount()
potassium_time = potassium_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
potassium_time = potassium_time.iloc[:,:length_potassium]
potassium_time = potassium_time.fillna(1000000)
column = []
i = 0
while i < length_potassium:
    column.append("potassium_offset_" + "%s"%str(i))
    i += 1
potassium_time.columns = column

i = 1
potassium_delta = potassium
potassium_delta = pd.merge(potassium_delta,potassium_time,on='patientunitstayid',how='left')
while i < length_potassium:
    potassium_delta['potassium_delta_%s'%str(i)] = (potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_%s'%str(i)] - potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_%s'%str(i-1)]) \
    / ((potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_offset_%s'%str(i)] - potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_potassium:
    potassium_delta.drop(columns = ['potassium_%s'%str(i)], inplace=True)
    potassium_delta.drop(columns = ['potassium_offset_%s'%str(i)], inplace=True)
    i += 1
potassium_delta = potassium_delta.fillna(1000000)
potassium_delta = pd.DataFrame(potassium_delta,dtype = np.float)
for i in potassium[(potassium['potassium_0'] != 1000000) & (potassium['potassium_0'] > 10)].index.values:
    potassium.loc[i]['potassium_0'] = 1000000
for i in potassium[(potassium['potassium_1'] != 1000000) & (potassium['potassium_1'] > 10)].index.values:
    potassium.loc[i]['potassium_1'] = 1000000
aki_potassium = potassium
aki_potassium_delta = potassium_delta

print("potassium delta 1 Missing Rate:",1 - potassium_delta[potassium_delta['potassium_delta_1']!=1000000]['potassium_delta_1'].count()/19615)
print(potassium,potassium_delta)


potassium max columns:38
potassium avg columns:2.3442622950819674
potassium missing rate:0.15100688248789193
potassium delta 1 Missing Rate: 0.487891919449401
                   potassium_0  potassium_1
patientunitstayid                          
141266                     5.2    1000000.0
141304                     4.8          5.8
141314                     8.0    1000000.0
141448                     2.8    1000000.0
141470                     4.3    1000000.0
...                        ...          ...
3353190                    4.2          4.6
3353194                    4.7          3.2
3353197                    4.3          4.2
3353251                    3.9          3.9
3353254                    5.0    1000000.0

[16653 rows x 2 columns]                    potassium_delta_1
patientunitstayid                   
141266                1000000.000000
141304                      0.272727
141314                1000000.000000
141448                1000000.000000
141470               

In [38]:
#sodium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('sodium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
sodium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','labresult'])
sodium_res = sodium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
sodium_size = sodium_res.groupby("patientunitstayid").size()
print("sodium max columns:" + str(max(sodium_size)) + "\n" + "sodium avg columns:" + str(sum(sodium_size)/len(sodium_size)) + "\n" + "sodium missing rate:" + str((19615-len(sodium_size))/19615))

length_sodium = int(statistics.median(sodium_size))

#sodium turn continuous data from vertical to horizontal
sodium_res['cumCount'] = sodium_res.groupby(['patientunitstayid']).cumcount()
sodium = sodium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
sodium = sodium.iloc[:,:length_sodium]

#sodium let format right
column = []
i = 0
while i < length_sodium:
    column.append("sodium_" + "%s"%str(i))
    i += 1
sodium.columns = column
sodium = sodium.fillna(1000000)
sodium = pd.DataFrame(sodium,dtype = np.float)

#caculate delta

#store the measure offset
sodium_time = sodium_res
#sodium_time turn continuous data from vertical to horizontal
sodium_time['cumCount'] = sodium_time.groupby(['patientunitstayid']).cumcount()
sodium_time = sodium_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
sodium_time = sodium_time.iloc[:,:length_sodium]
sodium_time = sodium_time.fillna(1000000)
column = []
i = 0
while i < length_sodium:
    column.append("sodium_offset_" + "%s"%str(i))
    i += 1
sodium_time.columns = column

i = 1
sodium_delta = sodium
sodium_delta = pd.merge(sodium_delta,sodium_time,on='patientunitstayid',how='left')
while i < length_sodium:
    sodium_delta['sodium_delta_%s'%str(i)] = (sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_%s'%str(i)] - sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_%s'%str(i-1)]) \
    / ((sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_offset_%s'%str(i)] - sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_sodium:
    sodium_delta.drop(columns = ['sodium_%s'%str(i)], inplace=True)
    sodium_delta.drop(columns = ['sodium_offset_%s'%str(i)], inplace=True)
    i += 1
sodium_delta = sodium_delta.fillna(1000000)
sodium_delta = pd.DataFrame(sodium_delta,dtype = np.float)
for i in sodium[(sodium['sodium_0'] != 1000000) & (sodium['sodium_0'] > 180)].index.values:
    sodium.loc[i]['sodium_0'] = 1000000
for i in sodium[(sodium['sodium_1'] != 1000000) & (sodium['sodium_1'] > 180)].index.values:
    sodium.loc[i]['sodium_1'] = 1000000
for i in sodium[(sodium['sodium_0'] != 1000000) & (sodium['sodium_0'] < 100)].index.values:
    sodium.loc[i]['sodium_0'] = 1000000
for i in sodium[(sodium['sodium_1'] != 1000000) & (sodium['sodium_1'] < 100)].index.values:
    sodium.loc[i]['sodium_1'] = 1000000
aki_sodium = sodium
aki_sodium_delta = sodium_delta

print("sodium delta Missing Rate:",1 - sodium_delta[sodium_delta['sodium_delta_1']!=1000000]['sodium_delta_1'].count()/19615)
print(sodium,sodium_delta)


sodium max columns:38
sodium avg columns:2.2531145184475325
sodium missing rate:0.14881468264083608
sodium delta Missing Rate: 0.49824114198317615
                   sodium_0   sodium_1
patientunitstayid                     
141266                135.0  1000000.0
141304                141.0      140.0
141314                158.0  1000000.0
141448                138.0  1000000.0
141470                131.0  1000000.0
...                     ...        ...
3353190               139.0      141.0
3353194               137.0      140.0
3353197               134.0      132.0
3353251               136.0  1000000.0
3353254               141.0  1000000.0

[16696 rows x 2 columns]                    sodium_delta_1
patientunitstayid                
141266             1000000.000000
141304                  -0.272727
141314             1000000.000000
141448             1000000.000000
141470             1000000.000000
...                           ...
3353190                  0.159574
3353194       

In [39]:
#Hct
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('Hct') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
Hct_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
Hct_res = Hct_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
Hct_size = Hct_res.groupby("patientunitstayid").size()
print("Hct max columns:" + str(max(Hct_size)) + "\n" + "Hct avg columns:" + str(sum(Hct_size) / len(Hct_size)) + "\n" + "Hct missing rate:" + str((19615-len(Hct_size))/19615))

length_Hct = int(statistics.median(Hct_size))

#Hct turn continuous data from vertical to horizontal
Hct_res['cumCount'] = Hct_res.groupby(['patientunitstayid']).cumcount()
Hct = Hct_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
Hct = Hct.iloc[:,:length_Hct]

#Hct let format right
column = []
i = 0
while i < length_Hct:
    column.append("Hct_" + "%s"%str(i))
    i += 1
Hct.columns = column
Hct = Hct.fillna(1000000)
Hct = pd.DataFrame(Hct,dtype = np.float)

#caculate delta

#store the measure offset
Hct_time = Hct_res
#Hct_time turn continuous data from vertical to horizontal
Hct_time['cumCount'] = Hct_time.groupby(['patientunitstayid']).cumcount()
Hct_time = Hct_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
Hct_time = Hct_time.iloc[:,:length_Hct]
Hct_time = Hct_time.fillna(1000000)
column = []
i = 0
while i < length_Hct:
    column.append("Hct_offset_" + "%s"%str(i))
    i += 1
Hct_time.columns = column

i = 1
Hct_delta = Hct
Hct_delta = pd.merge(Hct_delta,Hct_time,on='patientunitstayid',how='left')
while i < length_Hct:
    Hct_delta['Hct_delta_%s'%str(i)] = (Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_%s'%str(i)] - Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_%s'%str(i-1)]) \
    / ((Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_offset_%s'%str(i)] - Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_Hct:
    Hct_delta.drop(columns = ['Hct_%s'%str(i)], inplace=True)
    Hct_delta.drop(columns = ['Hct_offset_%s'%str(i)], inplace=True)
    i += 1
Hct_delta = Hct_delta.fillna(1000000)
Hct_delta = pd.DataFrame(Hct_delta,dtype = np.float)
for i in Hct[(Hct['Hct_0'] != 1000000) & (Hct['Hct_0'] > 70)].index.values:
    Hct.loc[i]['Hct_0'] = 1000000
for i in Hct[(Hct['Hct_1'] != 1000000) & (Hct['Hct_1'] > 70)].index.values:
    Hct.loc[i]['Hct_1'] = 1000000
for i in Hct[(Hct['Hct_0'] != 1000000) & (Hct['Hct_0'] < 15)].index.values:
    Hct.loc[i]['Hct_0'] = 1000000
for i in Hct[(Hct['Hct_1'] != 1000000) & (Hct['Hct_1'] < 15)].index.values:
    Hct.loc[i]['Hct_1'] = 1000000
aki_Hct = Hct
aki_Hct_delta = Hct_delta

print("Hct delta Missing Rate:",1 - Hct_delta[Hct_delta['Hct_delta_1']!=1000000]['Hct_delta_1'].count()/19615)
print(Hct,Hct_delta)


Hct max columns:32
Hct avg columns:1.9812522975125597
Hct missing rate:0.16788172317104258
Hct delta Missing Rate: 0.5792505735406577
                   Hct_0      Hct_1
patientunitstayid                  
141266              33.4  1000000.0
141304              28.8       25.5
141314              34.1  1000000.0
141448              35.8  1000000.0
141470              42.8  1000000.0
...                  ...        ...
3353190             34.0       33.0
3353194             36.0       35.0
3353197             23.0       23.0
3353251             33.0  1000000.0
3353254             38.0       33.0

[16322 rows x 2 columns]                       Hct_delta_1
patientunitstayid                
141266             1000000.000000
141304                  -0.900000
141314             1000000.000000
141448             1000000.000000
141470             1000000.000000
...                           ...
3353190                 -0.079787
3353194                 -0.164384
3353197                  0.00000

In [40]:
#BUN
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('BUN') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
BUN_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
BUN_res = BUN_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
BUN_size = BUN_res.groupby("patientunitstayid").size()
print("BUN max columns:" + str(max(BUN_size)) + "\n" + "BUN avg columns:" + str(sum(BUN_size)/len(BUN_size)) + "\n" + "BUN missing rate:" + str((19615-len(BUN_size))/19615))

length_BUN = int(statistics.median(BUN_size))

#BUN turn continuous data from vertical to horizontal
BUN_res['cumCount'] = BUN_res.groupby(['patientunitstayid']).cumcount()
BUN = BUN_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
BUN = BUN.iloc[:,:length_BUN]

#BUN let format right
column = []
i = 0
while i < length_BUN:
    column.append("BUN_" + "%s"%str(i))
    i += 1
BUN.columns = column
BUN = BUN.fillna(1000000)
BUN = pd.DataFrame(BUN,dtype = np.float)

#caculate delta

#store the measure offset
BUN_time = BUN_res
#BUN_time turn continuous data from vertical to horizontal
BUN_time['cumCount'] = BUN_time.groupby(['patientunitstayid']).cumcount()
BUN_time = BUN_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
BUN_time = BUN_time.iloc[:,:length_BUN]
BUN_time = BUN_time.fillna(1000000)
column = []
i = 0
while i < length_BUN:
    column.append("BUN_offset_" + "%s"%str(i))
    i += 1
BUN_time.columns = column

i = 1
BUN_delta = BUN
BUN_delta = pd.merge(BUN_delta,BUN_time,on='patientunitstayid',how='left')
while i < length_BUN:
    BUN_delta['BUN_delta_%s'%str(i)] = (BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_%s'%str(i)] - BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_%s'%str(i-1)]) \
    / ((BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_offset_%s'%str(i)] - BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_BUN:
    BUN_delta.drop(columns = ['BUN_%s'%str(i)], inplace=True)
    BUN_delta.drop(columns = ['BUN_offset_%s'%str(i)], inplace=True)
    i += 1
BUN_delta = BUN_delta.fillna(1000000)
BUN_delta = pd.DataFrame(BUN_delta,dtype = np.float)
for i in BUN[(BUN['BUN_0'] != 1000000) & (BUN['BUN_0'] > 70)].index.values:
    BUN.loc[i]['BUN_0'] = 1000000
for i in BUN[(BUN['BUN_1'] != 1000000) & (BUN['BUN_1'] > 70)].index.values:
    BUN.loc[i]['BUN_0'] = 1000000
for i in BUN[(BUN['BUN_0'] != 1000000) & (BUN['BUN_0'] < 1)].index.values:
    BUN.loc[i]['BUN_0'] = 1000000
for i in BUN[(BUN['BUN_1'] != 1000000) & (BUN['BUN_1'] < 1)].index.values:
    BUN.loc[i]['BUN_1'] = 1000000
aki_BUN = BUN
aki_BUN_delta = BUN_delta

print("BUN delta Missing Rate:",1 - BUN_delta[BUN_delta['BUN_delta_1']!=1000000]['BUN_delta_1'].count()/19615)
print(BUN,BUN_delta)


BUN max columns:19
BUN avg columns:2.017347307553307
BUN missing rate:0.15360693346928372
BUN delta Missing Rate: 0.5236298750955901
                   BUN_0      BUN_1
patientunitstayid                  
141266              29.0  1000000.0
141304              41.0       48.0
141314              13.0  1000000.0
141448              31.0  1000000.0
141470              33.0  1000000.0
...                  ...        ...
3353190             19.0       22.0
3353194             21.0       24.0
3353197             49.0       57.0
3353251             19.0  1000000.0
3353254             42.0  1000000.0

[16602 rows x 2 columns]                       BUN_delta_1
patientunitstayid                
141266             1000000.000000
141304                   1.909091
141314             1000000.000000
141448             1000000.000000
141470             1000000.000000
...                           ...
3353190                  0.239362
3353194                  0.493151
3353197                  0.337553

In [41]:
#pH
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('pH') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
pH_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
pH_res = pH_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
pH_size = pH_res.groupby("patientunitstayid").size()
print("pH max columns:" + str(max(pH_size)) + "\n" + "pH avg columns:" + str(sum(pH_size)/len(pH_size)) + "\n" + "pH missing rate:" + str((19615-len(pH_size))/19615))

length_pH = int(statistics.median(pH_size)) + 1

#pH turn continuous data from vertical to horizontal
pH_res['cumCount'] = pH_res.groupby(['patientunitstayid']).cumcount()
pH = pH_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
pH = pH.iloc[:,:length_pH]

#pH let format right
column = []
i = 0
while i < length_pH:
    column.append("pH_" + "%s"%str(i))
    i += 1
pH.columns = column
pH = pH.fillna(1000000)
pH = pd.DataFrame(pH,dtype = np.float)

#caculate delta

#store the measure offset
pH_time = pH_res
#pH_time turn continuous data from vertical to horizontal
pH_time['cumCount'] = pH_time.groupby(['patientunitstayid']).cumcount()
pH_time = pH_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
pH_time = pH_time.iloc[:,:length_pH]
pH_time = pH_time.fillna(1000000)
column = []
i = 0
while i < length_pH:
    column.append("pH_offset_" + "%s"%str(i))
    i += 1
pH_time.columns = column

i = 1
pH_delta = pH
pH_delta = pd.merge(pH_delta,pH_time,on='patientunitstayid',how='left')
while i < length_pH:
    pH_delta['pH_delta_%s'%str(i)] = (pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_%s'%str(i)] - pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_%s'%str(i-1)]) \
    / ((pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_offset_%s'%str(i)] - pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_pH:
    pH_delta.drop(columns = ['pH_%s'%str(i)], inplace=True)
    pH_delta.drop(columns = ['pH_offset_%s'%str(i)], inplace=True)
    i += 1
pH_delta = pH_delta.fillna(1000000)
pH_delta = pd.DataFrame(pH_delta,dtype = np.float)
for i in pH[(pH['pH_0'] != 1000000) & (pH['pH_0'] > 7.8)].index.values:
    pH.loc[i]['pH_0'] = 1000000
for i in pH[(pH['pH_1'] != 1000000) & (pH['pH_1'] > 7.8)].index.values:
    pH.loc[i]['pH_1'] = 1000000
for i in pH[(pH['pH_0'] != 1000000) & (pH['pH_0'] < 6.8)].index.values:
    pH.loc[i]['pH_0'] = 1000000
for i in pH[(pH['pH_1'] != 1000000) & (pH['pH_1'] < 6.8)].index.values:
    pH.loc[i]['pH_1'] = 1000000
aki_pH = pH
aki_pH_delta = pH_delta

print("pH delta Missing Rate:",1 - pH_delta[pH_delta['pH_delta_1']!=1000000]['pH_delta_1'].count()/19615)
print(pH,pH_delta)


pH max columns:36
pH avg columns:2.401624548736462
pH missing rate:0.5481009431557482
pH delta Missing Rate: 0.7790976293652817
                    pH_0         pH_1
patientunitstayid                    
141266             7.180        7.270
141304             7.320  1000000.000
141314             7.140        7.310
141515             7.170  1000000.000
141892             7.310        7.470
...                  ...          ...
3353144            7.444  1000000.000
3353190            7.308  1000000.000
3353194            7.156        7.269
3353197            7.304        7.381
3353251            6.973        7.182

[8864 rows x 2 columns]                        pH_delta_1
patientunitstayid                
141266                   0.083077
141304             1000000.000000
141314                   0.248780
141515             1000000.000000
141892                   0.021007
...                           ...
3353144            1000000.000000
3353190            1000000.000000
3353194      

In [42]:
#OI
#paO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('paO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
paO2_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','paO2_labresult'])
paO2_res = paO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'paO2_labresult':np.min}).reset_index()
paO2_size = paO2_res.groupby("patientunitstayid").size()
print("paO2 max columns:" + str(max(paO2_size)) + "\n" + "paO2 avg columns:" + str(sum(paO2_size)/len(paO2_size)) + "\n" + "paO2 missing rate:" + str((19615-len(paO2_size))/19615))

#FiO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('FiO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
FiO2_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','FiO2_labresult'])
FiO2_res = FiO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'FiO2_labresult':np.min}).reset_index()
FiO2_size = FiO2_res.groupby("patientunitstayid").size()
print("FiO2 max columns:" + str(max(FiO2_size)) + "\n" + "FiO2 avg columns:" + str(sum(FiO2_size)/len(FiO2_size)) + "\n" + "FiO2 missing rate:" + str((19615-len(FiO2_size))/19615))

#pick FiO2 and paO2 at the same time,can use this dataframe to calculate ratio
OI = pd.DataFrame()
OI = pd.merge(paO2_res,FiO2_res,on = ['patientunitstayid','labresultoffset'],how = 'inner')
OI_size = OI.groupby("patientunitstayid").size()
print("paO2/FiO2 max columns:" + str(max(OI_size)) + "\n" + "paO2/FiO2 avg columns:" + str(sum(OI_size)/len(OI_size)) + "\n" + "paO2/FiO2 missing rate:" + str((19615-len(OI_size))/19615))

length_OI = int(statistics.median(OI_size))

i = 0
clean_z = OI['FiO2_labresult'].fillna(1000000)
clean_z[clean_z == ''] = 1000000
OI['FiO2_labresult'] = clean_z
clean_z = OI['FiO2_labresult'].fillna(1000000)
clean_z[clean_z ==  0] = 1000000
OI['FiO2_labresult'] = clean_z
OI['OI'] = OI['paO2_labresult'] / OI['FiO2_labresult']
OI.drop(columns = ['paO2_labresult','FiO2_labresult'], inplace = True)

#OI turn continuous data from vertical to horizontal
OI['cumCount'] = OI.groupby(['patientunitstayid']).cumcount()
OI = OI.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'OI')
OI = OI.iloc[:,:length_OI]

#OI let format right
column = []
i = 0
while i < length_OI:
    column.append("OI_"+"%s"%str(i))
    i += 1
OI.columns = column
OI = OI.fillna(1000000)
OI = pd.DataFrame(OI,dtype = np.float)
# for i in OI[(OI['OI_0'] != 1000000) & (OI['OI_0'] > 1000)].index.values:
#     OI.loc[i]['OI_0'] = 1000000
# for i in OI[(OI['OI_0'] != 1000000) & (OI['OI_0'] < 50)].index.values:
#     OI.loc[i]['OI_0'] = 1000000
aki_OI = OI

print("OI 0 Missing Rate:",1 - OI[OI['OI_0']!=1000000]['OI_0'].count()/19615)
print(OI)


paO2 max columns:36
paO2 avg columns:2.4001795533610144
paO2 missing rate:0.5457048177415244
FiO2 max columns:30
FiO2 avg columns:2.217826501429933
FiO2 missing rate:0.5721641600815702
paO2/FiO2 max columns:30
paO2/FiO2 avg columns:2.260608598548297
paO2/FiO2 missing rate:0.6347693092021413
OI 0 Missing Rate: 0.637726229926077
                         OI_0
patientunitstayid            
141266               2.910000
141314               0.737500
141515               2.830000
141892               2.933333
142631               5.300000
...                       ...
3353144              3.890000
3353190              3.708333
3353194              0.680000
3353197            226.000000
3353251            246.000000

[7164 rows x 1 columns]


In [43]:
#temperature
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Temperature (C)') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
TP_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','nursingchartvalue'])
TP_size = TP_res.groupby("patientunitstayid").size()
print("Temperature max columns:" + str(max(TP_size)) + "\n" + "Temperature avg columns:" + str(sum(TP_size) / len(TP_size)) + "\n" + "Temperature missing rate:" + str((19615-len(TP_size))/19615))

length_TP = int(statistics.median(TP_size))

#store the measure offset
TP_time = TP_res
#TP_time turn continuous data from vertical to horizontal
TP_time['cumCount'] = TP_time.groupby(['patientunitstayid']).cumcount()
TP_time = TP_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')
TP_time = TP_time.iloc[:,:length_TP]

#TP turn continuous data from vertical to horizontal
TP_res['cumCount'] = TP_res.groupby(['patientunitstayid']).cumcount()
TP = TP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartvalue')
TP = TP.iloc[:,:length_TP]

#find extreme and slope

#TP let format right
column = []
i = 0
while i < length_TP:
    column.append("TP_" + "%s"%str(i))
    i += 1
TP.columns = column
TP.fillna(1000000)
TP = pd.DataFrame(TP,dtype = np.float)

#find extreme
TP_time.columns = column
TP_sum = pd.DataFrame()
TP_sum['max_TP'] = TP.max(axis=1)
TP_sum['max_INDEX'] = TP.idxmax(axis=1)
TP_sum['min_TP'] = TP.min(axis=1,skipna = True)
TP_sum['min_INDEX'] = TP.idxmin(axis=1, skipna=True)
TP_sum['TP_delta'] = TP_sum['max_TP'] - TP_sum['min_TP']
TP_sum['TP_fever'] = 1*(TP_sum['max_TP'] >= 38)
ind = 0
TP_sum['slope'] = 0
for i in TP_sum.index:
    during = TP_time.loc[i,TP_sum['max_INDEX'].loc[i]] - TP_time.loc[i,TP_sum['min_INDEX'].loc[i]]
    during /= 60
    if during != 0:
        TP_sum['slope'].iloc[ind] = TP_sum['TP_delta'].iloc[ind] / during
    else:
        TP_sum['slope'].iloc[ind] = 1000000
    ind += 1
TP_sum.drop('max_INDEX',axis=1,inplace = True)
TP_sum.drop('min_INDEX',axis=1,inplace = True)
TP = TP.fillna(1000000)
TP_sum = TP_sum.fillna(1000000)

#caculate delta
TP_time.rename(columns={'TP_0':'TP_offset_0', 'TP_1':'TP_offset_1','TP_2':'TP_offset_2', 'TP_3':'TP_offset_3','TP_4':'TP_offset_4', 'TP_5':'TP_offset_5'}, inplace=True)
TP_time.fillna(1000000)
TP_delta = TP
TP_delta = pd.merge(TP_delta,TP_time,on='patientunitstayid',how='left')
i = 1
while i < length_TP:
    TP_delta['TP_delta_%s'%str(i)] = (TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_%s'%str(i)] - TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_%s'%str(i-1)]) \
    / ((TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_offset_%s'%str(i)] - TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_TP:
    TP_delta.drop(columns = ['TP_%s'%str(i)], inplace=True)
    TP_delta.drop(columns = ['TP_offset_%s'%str(i)], inplace=True)
    i += 1
TP_delta = TP_delta.fillna(1000000)
TP_delta = pd.DataFrame(TP_delta,dtype = np.float)

aki_TP = TP
aki_TP_delta = TP_delta
aki_TP_sum = TP_sum

print("TP delta Missing Rate:",1 - TP_sum[TP_sum['TP_delta']!=1000000]['TP_delta'].count()/19615)
print("TP delta 1 Missing Rate:",1 - TP_delta[TP_delta['TP_delta_1']!=1000000]['TP_delta_1'].count()/19615)
print(TP,TP_delta,TP_sum)


Temperature max columns:219
Temperature avg columns:7.13069586718474
Temperature missing rate:0.13403007902115727
TP delta Missing Rate: 0.13403007902115727
TP delta 1 Missing Rate: 0.39979607443283205
                   TP_0       TP_1       TP_2
patientunitstayid                            
141266             36.3       36.2  1000000.0
141304             36.8       38.5       38.0
141314             35.5       35.4       35.2
141448             36.8       37.5       36.6
141470             36.7       37.9       38.2
...                 ...        ...        ...
3353194            35.4       34.4       34.4
3353197            38.2       38.2       38.7
3353199            37.1  1000000.0  1000000.0
3353251            33.4       33.3       33.2
3353254            36.5       37.1       36.4

[16986 rows x 3 columns]                        TP_delta_1      TP_delta_2
patientunitstayid                                
141266                  -0.023622  1000000.000000
141304                  

In [44]:
print("TP delta 2 Missing Rate:",1 - TP_delta[TP_delta['TP_delta_2']!=1000000]['TP_delta_2'].count()/19615)


TP delta 2 Missing Rate: 0.5625796584246749


In [45]:
#urine
i = 0
offset = 360
NextOffset = 0
urine_dflist = []
urine_tdflist = []
for i in range(1,1 + int(int(strsql_timeoffst)/360)):
    PreOffset = NextOffset
    NextOffset = offset*i
    sql = "SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.cellvaluenumeric FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.celllabel IN('Urine') AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY IO.patientunitstayid,IO.intakeoutputoffset"
    cur.execute(sql)
    urine_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','intakeoutputoffset_'+str(i),'urine_'+str(i)])
    urine_tdflist.append(urine_res.groupby(['patientunitstayid']).agg({'intakeoutputoffset_'+str(i):np.sum}).reset_index())
    urine_res = urine_res.groupby(['patientunitstayid']).agg({'urine_'+str(i):np.sum}).reset_index() 
    urine_dflist.append(urine_res)


urine = pd.DataFrame()
urine = urine_dflist[0]
for data in urine_dflist[1:]:
    urine = pd.merge(urine,data,on = 'patientunitstayid',how = 'left')
urine = urine.set_index('patientunitstayid')
urine = urine.fillna(1000000)
# print(len(urine.index))
i = 0
total = []
while i < len(urine.index):
    a = 0
    if urine.iloc[i]['urine_1'] != 1000000:
        a += float(urine.iloc[i]['urine_1'])
    if urine.iloc[i]['urine_2'] != 1000000:
        a += float(urine.iloc[i]['urine_2'])
    if urine.iloc[i]['urine_3'] != 1000000:
        a += float(urine.iloc[i]['urine_3'])
    if urine.iloc[i]['urine_4'] != 1000000:
        a += float(urine.iloc[i]['urine_4'])
    total.append(a)
    i += 1
urine['urine_total'] = total
urine = pd.DataFrame(urine,dtype = np.float)

# #caculate delta
# urine_time = pd.DataFrame()
# urine_time = urine_tdflist[0]
# for data in urine_tdflist[1:]:
#     urine_time = pd.merge(urine_time,data,on = 'patientunitstayid',how = 'left')
# urine_time = urine_time.set_index('patientunitstayid')
# urine_time = urine_time.fillna(1000000)
# urine_time = pd.DataFrame(urine_time,dtype = np.float)

# i = 2
# urine_delta = urine
# urine_delta = pd.merge(urine_delta,urine_time,on='patientunitstayid',how='left')
# while i < 5:
#     urine_delta['urine_delta_%s'%str(i-1)] = (urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['urine_%s'%str(i)] - urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['urine_%s'%str(i-1)]) \
#     / ((urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['intakeoutputoffset_%s'%str(i)] - urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['intakeoutputoffset_%s'%str(i-1)])/60)
#     i += 1
# i = 1
# while i < 5:
#     urine_delta.drop(columns = ['urine_%s'%str(i)], inplace=True)
#     urine_delta.drop(columns = ['intakeoutputoffset_%s'%str(i)], inplace=True)
#     i += 1
# urine_delta = urine_delta.fillna(1000000)
# urine_delta = pd.DataFrame(urine_delta,dtype = np.float)
urine.drop(columns = ['urine_1','urine_2','urine_3','urine_4'], inplace=True)
# urine_delta.drop(columns = ['urine_total'], inplace=True)
aki_urine = urine
# aki_urine_delta = urine_delta

print("urine total Missing Rate:",1 - urine[urine['urine_total']!=1000000]['urine_total'].count()/19615)
print(urine)
print(urine[urine['urine_total'] > 600]['urine_total'].count())


urine total Missing Rate: 0.6330359418812134
                     urine_5    urine_6    urine_7    urine_8  urine_total
patientunitstayid                                                         
141594             1000000.0  1000000.0  1000000.0  1000000.0        300.0
141786             1000000.0  1000000.0  1000000.0  1000000.0        200.0
141866             1000000.0  1000000.0  1000000.0  1000000.0        600.0
141893             1000000.0  1000000.0  1000000.0  1000000.0        310.0
142129             1000000.0  1000000.0  1000000.0  1000000.0       2000.0
...                      ...        ...        ...        ...          ...
3353190            1000000.0  1000000.0  1000000.0  1000000.0        400.0
3353194                 21.0      135.0       40.0  1000000.0        455.0
3353197                220.0      270.0      600.0      125.0        890.0
3353199            1000000.0  1000000.0  1000000.0  1000000.0         75.0
3353251            1000000.0  1000000.0  1000000.0  100

In [46]:
#diastolic blood pressure(noninvasive)
cur.execute("SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivediastolic FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,observationoffset")
diastolicBP_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','observationoffset','noninvasivediastolic'])
diastolicBP_res = diastolicBP_res.groupby(['patientunitstayid','observationoffset']).agg({'noninvasivediastolic':np.min}).reset_index()
diastolicBP_size = diastolicBP_res.groupby("patientunitstayid").size()
print("diastolicBP max columns:" + str(max(diastolicBP_size)) + "\n" + "diastolicBP avg columns:" + str(sum(diastolicBP_size)/len(diastolicBP_size)) + "\n" + "diastolicBP missing rate:" + str((19615-len(diastolicBP_size))/19615))

length_diastolicBP = int(statistics.median(diastolicBP_size))

#diastolicBP turn continuous data from vertical to horizontal
diastolicBP_res['cumCount'] = diastolicBP_res.groupby(['patientunitstayid']).cumcount()
diastolicBP = diastolicBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'noninvasivediastolic')
diastolicBP = diastolicBP.iloc[:,:length_diastolicBP]

#diastolicBP let format right
column = []
i = 0
while i < length_diastolicBP:
    column.append("diastolicBP_"+"%s"%str(i))
    i += 1
diastolicBP.columns = column
diastolicBP = diastolicBP.fillna(1000000)
diastolicBP = pd.DataFrame(diastolicBP,dtype = np.float)

#caculate delta

#store the measure offset
diastolicBP_time = diastolicBP_res
#diastolicBP_time turn continuous data from vertical to horizontal
diastolicBP_time['cumCount'] = diastolicBP_time.groupby(['patientunitstayid']).cumcount()
diastolicBP_time = diastolicBP_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'observationoffset')
diastolicBP_time = diastolicBP_time.iloc[:,:length_diastolicBP]
diastolicBP_time = diastolicBP_time.fillna(1000000)
column = []
i = 0
while i < length_diastolicBP:
    column.append("diastolicBP_offset_" + "%s"%str(i))
    i += 1
diastolicBP_time.columns = column

i = 1
diastolicBP_delta = diastolicBP
diastolicBP_delta = pd.merge(diastolicBP_delta,diastolicBP_time,on='patientunitstayid',how='left')
while i < length_diastolicBP:
    diastolicBP_delta['diastolicBP_delta_%s'%str(i)] = (diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_%s'%str(i)] - diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_%s'%str(i-1)]) \
    / ((diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_offset_%s'%str(i)] - diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_diastolicBP:
    diastolicBP_delta.drop(columns = ['diastolicBP_%s'%str(i)], inplace=True)
    diastolicBP_delta.drop(columns = ['diastolicBP_offset_%s'%str(i)], inplace=True)
    i += 1
diastolicBP_delta = diastolicBP_delta.fillna(1000000)
diastolicBP_delta = pd.DataFrame(diastolicBP_delta,dtype = np.float)
aki_diastolicBP = diastolicBP
aki_diastolicBP_delta = diastolicBP_delta

print("diastolicBP delta 1 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_1']!=1000000]['diastolicBP_delta_1'].count()/19615)
print("diastolicBP delta 2 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_2']!=1000000]['diastolicBP_delta_2'].count()/19615)
print("diastolicBP delta 3 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_3']!=1000000]['diastolicBP_delta_3'].count()/19615)
print("diastolicBP delta 4 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_4']!=1000000]['diastolicBP_delta_4'].count()/19615)
print("diastolicBP delta 5 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_5']!=1000000]['diastolicBP_delta_5'].count()/19615)
print("diastolicBP delta 6 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_6']!=1000000]['diastolicBP_delta_6'].count()/19615)
print("diastolicBP delta 7 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_7']!=1000000]['diastolicBP_delta_7'].count()/19615)
print(diastolicBP,diastolicBP_delta)


diastolicBP max columns:2741
diastolicBP avg columns:25.96370064820271
diastolicBP missing rate:0.13484578128982921
diastolicBP delta 1 Missing Rate: 0.22900841192964572
diastolicBP delta 2 Missing Rate: 0.3177160336477186
diastolicBP delta 3 Missing Rate: 0.3837879174101453
diastolicBP delta 4 Missing Rate: 0.4392556716798368
diastolicBP delta 5 Missing Rate: 0.4804996176395615
diastolicBP delta 6 Missing Rate: 0.5135865409125668
diastolicBP delta 7 Missing Rate: 0.5433596737190926
                   diastolicBP_0  diastolicBP_1  diastolicBP_2  diastolicBP_3  \
patientunitstayid                                                               
141266                      63.0      1000000.0      1000000.0      1000000.0   
141304                      55.0           55.0           66.0           59.0   
141314                      37.0           39.0           38.0           48.0   
141448                      64.0      1000000.0      1000000.0      1000000.0   
141470                    

In [47]:
#systolic blood pressure(noninvasive)
cur.execute("SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivesystolic FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,observationoffset")
systolicBP_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','observationoffset','noninvasivesystolic'])
systolicBP_res = systolicBP_res.groupby(['patientunitstayid','observationoffset']).agg({'noninvasivesystolic':np.min}).reset_index()
systolicBP_size = systolicBP_res.groupby("patientunitstayid").size()
print("systolicBP max columns:" + str(max(systolicBP_size)) + "\n" + "systolicBP avg columns:" + str(sum(systolicBP_size)/len(systolicBP_size)) + "\n" + "systolicBP missing rate:" + str((19615-len(systolicBP_size))/19615))

length_systolicBP = int(statistics.median(systolicBP_size))

#systolicBP turn continuous data from vertical to horizontal
systolicBP_res['cumCount'] = systolicBP_res.groupby(['patientunitstayid']).cumcount()
systolicBP = systolicBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'noninvasivesystolic')
systolicBP = systolicBP.iloc[:,:length_systolicBP]

#systolicBP let format right
column = []
i = 0
while i < length_systolicBP:
    column.append("systolicBP_"+"%s"%str(i))
    i += 1
systolicBP.columns = column
systolicBP = systolicBP.fillna(1000000)
systolicBP = pd.DataFrame(systolicBP,dtype = np.float)

#caculate delta

#store the measure offset
systolicBP_time = systolicBP_res
#systolicBP_time turn continuous data from vertical to horizontal
systolicBP_time['cumCount'] = systolicBP_time.groupby(['patientunitstayid']).cumcount()
systolicBP_time = systolicBP_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'observationoffset')
systolicBP_time = systolicBP_time.iloc[:,:length_systolicBP]
systolicBP_time = systolicBP_time.fillna(1000000)
column = []
i = 0
while i < length_systolicBP:
    column.append("systolicBP_offset_" + "%s"%str(i))
    i += 1
systolicBP_time.columns = column

i = 1
systolicBP_delta = systolicBP 
systolicBP_delta = pd.merge(systolicBP_delta,systolicBP_time,on='patientunitstayid',how='left')
while i < length_systolicBP:
    systolicBP_delta['systolicBP_delta_%s'%str(i)] = (systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_%s'%str(i)] - systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_%s'%str(i-1)]) \
    / ((systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i)] - systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_systolicBP:
    systolicBP_delta.drop(columns = ['systolicBP_%s'%str(i)], inplace=True)
    systolicBP_delta.drop(columns = ['systolicBP_offset_%s'%str(i)], inplace=True)
    i += 1
systolicBP_delta = systolicBP_delta.fillna(1000000)
systolicBP_delta = pd.DataFrame(systolicBP_delta,dtype = np.float)
aki_systolicBP = systolicBP
aki_systolicBP_delta = systolicBP_delta

print("systolicBP delta 1 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_1']!=1000000]['systolicBP_delta_1'].count()/19615)
print("systolicBP delta 2 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_2']!=1000000]['systolicBP_delta_2'].count()/19615)
print("systolicBP delta 3 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_3']!=1000000]['systolicBP_delta_3'].count()/19615)
print("systolicBP delta 4 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_4']!=1000000]['systolicBP_delta_4'].count()/19615)
print("systolicBP delta 5 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_5']!=1000000]['systolicBP_delta_5'].count()/19615)
print("systolicBP delta 6 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_6']!=1000000]['systolicBP_delta_6'].count()/19615)
print("systolicBP delta 7 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_7']!=1000000]['systolicBP_delta_7'].count()/19615)
print(systolicBP,systolicBP_delta)


systolicBP max columns:2741
systolicBP avg columns:25.96370064820271
systolicBP missing rate:0.13484578128982921
systolicBP delta 1 Missing Rate: 0.22885546775426968
systolicBP delta 2 Missing Rate: 0.31751210808055064
systolicBP delta 3 Missing Rate: 0.38353301045118526
systolicBP delta 4 Missing Rate: 0.43905174611266884
systolicBP delta 5 Missing Rate: 0.4803466734641856
systolicBP delta 6 Missing Rate: 0.513484578128983
systolicBP delta 7 Missing Rate: 0.5433596737190926
                   systolicBP_0  systolicBP_1  systolicBP_2  systolicBP_3  \
patientunitstayid                                                           
141266                    101.0     1000000.0     1000000.0     1000000.0   
141304                    106.0         119.0         109.0          93.0   
141314                     53.0          62.0          76.0          83.0   
141448                    117.0     1000000.0     1000000.0     1000000.0   
141470                     94.0     1000000.0     1000000.

In [48]:
#pulse pressure
pulseP_res = pd.DataFrame()
pulseP_res = pd.merge(diastolicBP_res,systolicBP_res,on = ['patientunitstayid','observationoffset'],how = 'inner')
pulseP_res['pulse pressure'] = pulseP_res['noninvasivesystolic'] - pulseP_res['noninvasivediastolic']
pulseP_size = pulseP_res.groupby("patientunitstayid").size()
print("pulse pressure max columns:" + str(max(pulseP_size)) + "\n" + "pulse pressure avg columns:" + str(sum(pulseP_size)/len(pulseP_size)) + "\n" + "pulse pressure missing rate:" + str((19615-len(pulseP_size))/19615))

length_pulseP = int(statistics.median(pulseP_size))

#pulseP turn continuous data from vertical to horizontal
pulseP_res['cumCount'] = pulseP_res.groupby(['patientunitstayid']).cumcount()
pulseP = pulseP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'pulse pressure')
pulseP = pulseP.iloc[:,:length_pulseP]

#pulseP let format right
column = []
i = 0
while i < length_pulseP:
    column.append("pulseP_"+"%s"%str(i))
    i += 1
pulseP.columns = column
pulseP = pulseP.fillna(1000000)
pulseP = pd.DataFrame(pulseP,dtype = np.float)
aki_pulseP = pulseP

print("pulseP 1 Missing Rate:",1 - pulseP[pulseP['pulseP_1']!=1000000]['pulseP_1'].count()/19615)
print("pulseP 2 Missing Rate:",1 - pulseP[pulseP['pulseP_2']!=1000000]['pulseP_2'].count()/19615)
print("pulseP 3 Missing Rate:",1 - pulseP[pulseP['pulseP_3']!=1000000]['pulseP_3'].count()/19615)
print("pulseP 4 Missing Rate:",1 - pulseP[pulseP['pulseP_4']!=1000000]['pulseP_4'].count()/19615)
print("pulseP 5 Missing Rate:",1 - pulseP[pulseP['pulseP_5']!=1000000]['pulseP_5'].count()/19615)
print("pulseP 6 Missing Rate:",1 - pulseP[pulseP['pulseP_6']!=1000000]['pulseP_6'].count()/19615)
print("pulseP 7 Missing Rate:",1 - pulseP[pulseP['pulseP_7']!=1000000]['pulseP_7'].count()/19615)
print(pulseP)


pulse pressure max columns:2741
pulse pressure avg columns:25.96370064820271
pulse pressure missing rate:0.13484578128982921
pulseP 1 Missing Rate: 0.22900841192964572
pulseP 2 Missing Rate: 0.3177160336477186
pulseP 3 Missing Rate: 0.3837879174101453
pulseP 4 Missing Rate: 0.4392556716798368
pulseP 5 Missing Rate: 0.4804996176395615
pulseP 6 Missing Rate: 0.5135865409125668
pulseP 7 Missing Rate: 0.5433596737190926
                   pulseP_0   pulseP_1   pulseP_2   pulseP_3   pulseP_4  \
patientunitstayid                                                         
141266                 38.0  1000000.0  1000000.0  1000000.0  1000000.0   
141304                 51.0       64.0       43.0       34.0       51.0   
141314                 16.0       23.0       38.0       35.0       81.0   
141448                 53.0  1000000.0  1000000.0  1000000.0  1000000.0   
141470                 39.0  1000000.0  1000000.0  1000000.0  1000000.0   
...                     ...        ...        ...      

In [49]:
#mean blood pressure(noninvasive)
meanBP = pulseP
meanBP = pd.merge(meanBP,diastolicBP,on='patientunitstayid',how='left')
i = 0
while i < length_diastolicBP:
    meanBP['meanBP_%s'%str(i)] = ((meanBP[meanBP['pulseP_%s'%str(i)] < 1000000]['pulseP_%s'%str(i)]/3) + meanBP[meanBP['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_%s'%str(i)])
    i += 1
i = 0
while i < length_systolicBP:
    meanBP.drop(columns = ['pulseP_%s'%str(i)], inplace=True)
    meanBP.drop(columns = ['diastolicBP_%s'%str(i)], inplace=True)
    i += 1

meanBP = meanBP.fillna(1000000)
#caculate delta

#store the measure offset
i = 1
meanBP_delta = meanBP
meanBP_delta = pd.merge(meanBP_delta,systolicBP_time,on='patientunitstayid',how='left')
while i < length_systolicBP:
    meanBP_delta['meanBP_delta_%s'%str(i)] = (meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['meanBP_%s'%str(i)] - meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['meanBP_%s'%str(i-1)]) \
    / ((meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i)] - meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_systolicBP:
    meanBP_delta.drop(columns = ['meanBP_%s'%str(i)], inplace=True)
    meanBP_delta.drop(columns = ['systolicBP_offset_%s'%str(i)], inplace=True)
    i += 1
meanBP_delta = meanBP_delta.fillna(1000000)
meanBP_delta = pd.DataFrame(meanBP_delta,dtype = np.float)
aki_meanBP = meanBP
aki_meanBP_delta = meanBP_delta

print("meanBP 0 Missing Rate:",1 - meanBP[meanBP['meanBP_0']!=1000000]['meanBP_0'].count()/19615)
print("meanBP delta 1 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_1']!=1000000]['meanBP_delta_1'].count()/19615)
print("meanBP delta 2 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_2']!=1000000]['meanBP_delta_2'].count()/19615)
print("meanBP delta 3 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_3']!=1000000]['meanBP_delta_3'].count()/19615)
print("meanBP delta 4 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_4']!=1000000]['meanBP_delta_4'].count()/19615)
print("meanBP delta 5 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_5']!=1000000]['meanBP_delta_5'].count()/19615)
print("meanBP delta 6 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_6']!=1000000]['meanBP_delta_6'].count()/19615)
print("meanBP delta 7 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_7']!=1000000]['meanBP_delta_7'].count()/19615)
print(meanBP,meanBP_delta)


meanBP 0 Missing Rate: 0.13515166964058123
meanBP delta 1 Missing Rate: 0.22900841192964572
meanBP delta 2 Missing Rate: 0.3177160336477186
meanBP delta 3 Missing Rate: 0.3837879174101453
meanBP delta 4 Missing Rate: 0.4392556716798368
meanBP delta 5 Missing Rate: 0.4804996176395615
meanBP delta 6 Missing Rate: 0.5135865409125668
meanBP delta 7 Missing Rate: 0.5433596737190926
                     meanBP_0        meanBP_1        meanBP_2        meanBP_3  \
patientunitstayid                                                               
141266              75.666667  1000000.000000  1000000.000000  1000000.000000   
141304              72.000000       76.333333       80.333333       70.333333   
141314              42.333333       46.666667       50.666667       59.666667   
141448              81.666667  1000000.000000  1000000.000000  1000000.000000   
141470              68.000000  1000000.000000  1000000.000000  1000000.000000   
...                       ...             ...        

In [50]:
#MSI modified shock index
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Heart Rate') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
HR_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','HR_nursingchartvalue'])
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Non-Invasive BP Systolic') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
SBP_res = pd.DataFrame(cur.fetchall(),columns =[ 'patientunitstayid','nursingchartoffset','SBP_nursingchartvalue'])
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Non-Invasive BP Diastolic') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
DBP_res = pd.DataFrame(cur.fetchall(),columns =[ 'patientunitstayid','nursingchartoffset','DBP_nursingchartvalue'])

# time offset
HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
HR_time = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
SBP_time = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

DBP_res['cumCount'] = DBP_res.groupby(['patientunitstayid']).cumcount()
DBP_time = DBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

#real value
HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
HR = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'HR_nursingchartvalue')

SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
SBP = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'SBP_nursingchartvalue')

DBP_res['cumCount'] = DBP_res.groupby(['patientunitstayid']).cumcount()
DBP = DBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'DBP_nursingchartvalue')

HR = HR.fillna(1000000)
SBP = SBP.fillna(1000000)
DBP = DBP.fillna(1000000)
count = 0

i = 0
column = []
while i < len(HR.columns):
    column.append("HR_%s"%i)
    i += 1
HR.columns = column
# print(HR)

i = 0
column = []
while i < len(SBP.columns):
    column.append("SBP_%s"%i)
    i += 1
SBP.columns = column
# print(SBP)

i = 0
column = []
while i < len(DBP.columns):
    column.append("DBP_%s"%i)
    i += 1
DBP.columns = column

HR_time = HR_time.fillna(1000000)
SBP_time = SBP_time.fillna(1000000)
DBP_time = DBP_time.fillna(1000000)
count = 0

i = 0
column = []
while i < len(HR_time.columns):
    column.append("HR_offset_%s"%i)
    i += 1
HR_time.columns = column
# print(HR)

i = 0
column = []
while i < len(SBP_time.columns):
    column.append("SBP_offset_%s"%i)
    i += 1
SBP_time.columns = column
# print(SBP)

i = 0
column = []
while i < len(DBP_time.columns):
    column.append("DBP_offset_%s"%i)
    i += 1
DBP_time.columns = column

DBP_size = DBP_res.groupby("patientunitstayid").size()
SBP_size = SBP_res.groupby("patientunitstayid").size()
HR_size = HR_res.groupby("patientunitstayid").size()

length_MSI = int(statistics.median(DBP_size))

MSI = []
i = 0
while i < len(DBP.index):
    #one time deal with one patient
    goal = 0
    loop = 0
#     whether index is in DBP SBP HR
    if DBP.index[i] in HR.index.values and DBP.index[i] in SBP.index.values:
        SIpt = []
        SIpt.append(DBP.index[i])
#         take DBP columns as basic columns length check loop value under length and success under 9
        while loop < DBP_size.loc[DBP.index[i]] and goal < length_MSI:
            #whether we have every ingredient
            if DBP_time.loc[DBP.index[i]][loop] in SBP_time.loc[DBP.index[i]].values and DBP_time.loc[DBP.index[i]][loop]in HR_time.loc[DBP.index[i]].values:
                DBP_col_index = loop
                HR_col_index = int(np.where(HR_time.loc[DBP.index[i]].values == DBP_time.loc[DBP.index[i]][loop])[0])
                SBP_col_index = int(np.where(SBP_time.loc[DBP.index[i]].values == DBP_time.loc[DBP.index[i]][loop])[0])
                #caculate MSI
                msi = 0
                pulse = 0
                MAP = 0
                if (float(HR.loc[DBP.index[i]][HR_col_index]) != 0 or 1000000) and (float(SBP.loc[DBP.index[i]][SBP_col_index]) != 0 or 1000000) and (float(DBP.loc[DBP.index[i]][DBP_col_index]) != 0 or 1000000): 
                    pulse = float(SBP.loc[DBP.index[i]][SBP_col_index]) - float(DBP.loc[DBP.index[i]][DBP_col_index])
                    MAP = pulse/3 + float(DBP.loc[DBP.index[i]][DBP_col_index])
                    if MAP != 0:
                        msi = float(HR.loc[DBP.index[i]][HR_col_index]) / MAP
                        SIpt.append(msi)
                        goal += 1
            loop += 1
        MSI.append(SIpt)
    i += 1

MSI = pd.DataFrame(MSI)
column = ['patientunitstayid']
i = 0
while i < length_MSI:
    column.append("MSI_%s"%i)
    i += 1
MSI.columns = column
MSI.set_index('patientunitstayid',inplace = True)
MSI = MSI.fillna(1000000)
MSI = pd.DataFrame(MSI,dtype = np.float)
aki_MSI = MSI

print("MSI 0 Missing Rate:",1 - MSI[MSI['MSI_0']!=1000000]['MSI_0'].count()/19615)
print("MSI 1 Missing Rate:",1 - MSI[MSI['MSI_1']!=1000000]['MSI_1'].count()/19615)
print("MSI 2 Missing Rate:",1 - MSI[MSI['MSI_2']!=1000000]['MSI_2'].count()/19615)
print("MSI 3 Missing Rate:",1 - MSI[MSI['MSI_3']!=1000000]['MSI_3'].count()/19615)
print("MSI 4 Missing Rate:",1 - MSI[MSI['MSI_4']!=1000000]['MSI_4'].count()/19615)
print("MSI 5 Missing Rate:",1 - MSI[MSI['MSI_5']!=1000000]['MSI_5'].count()/19615)
print("MSI 6 Missing Rate:",1 - MSI[MSI['MSI_6']!=1000000]['MSI_6'].count()/19615)
print("MSI 7 Missing Rate:",1 - MSI[MSI['MSI_7']!=1000000]['MSI_7'].count()/19615)
print(MSI)


MSI 0 Missing Rate: 0.17221514147336225
MSI 1 Missing Rate: 0.2884017333673209
MSI 2 Missing Rate: 0.362681621208259
MSI 3 Missing Rate: 0.4219729798623503
MSI 4 Missing Rate: 0.47484068315065
MSI 5 Missing Rate: 0.5183787917410145
MSI 6 Missing Rate: 0.5551873566148355
MSI 7 Missing Rate: 0.5890390007647208
                      MSI_0           MSI_1           MSI_2           MSI_3  \
patientunitstayid                                                             
141266             1.161972        1.050000        1.134021        1.137931   
141304             1.314607        1.258065        1.500000        1.513158   
141314             2.040000        2.142857        1.575419        1.214286   
141448             1.287129        1.351351        1.062857        1.139241   
141470             1.221154        1.567568        1.600000        1.647059   
...                     ...             ...             ...             ...   
3353190            1.046296        1.068966        1.16803

In [51]:
#age,since its missing rate is 0,so I don't print its relative information
cur.execute("SELECT patientunitstayid,age FROM patient WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
age_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','age'])
age = age_res
age = np.array(age)
age = age.tolist()
for i in age:
    if i[1].isdigit() == False:
        i[1] = ''.join([x for x in i[1] if x.isdigit()])
        if i[1] != '':
            i[1] = int(i[1])
        else:
            i[1] = 1000000
    else:
        i[1] = int(i[1])
age = pd.DataFrame(age)
age.columns = ['patientunitstayid','age']
age.set_index('patientunitstayid',inplace = True)
age = pd.DataFrame(age,dtype = np.float)
aki_age =age

print(age)


                    age
patientunitstayid      
141266             73.0
141304             70.0
141314             45.0
141448             65.0
141470             58.0
...                 ...
3353194            51.0
3353197            66.0
3353199            66.0
3353251            73.0
3353254            81.0

[19615 rows x 1 columns]


In [52]:
#weight
cur.execute("SELECT patientunitstayid,admissionweight FROM patient WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) AND admissionweight IS NOT NULL ORDER BY patientunitstayid")
weight_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','weight'])
weight_size = weight_res.groupby("patientunitstayid").size()
print("weight max columns:" + str(max(weight_size)) + "\n" + "weight avg columns:" + str(sum(weight_size)/len(weight_size)) + "\n" + "weight missing rate:" + str((19615-len(weight_size))/19615))

#weight turn continuous data from vertical to horizontal
weight_res['cumCount'] = weight_res.groupby(['patientunitstayid']).cumcount()
weight = weight_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'weight')


#weight let format right
weight = weight.fillna(1000000)
weight = pd.DataFrame(weight,dtype = np.float)
weight.columns = ['weight']
aki_weight = weight

print(weight)


weight max columns:1
weight avg columns:1.0
weight missing rate:0.04027529951567678
                   weight
patientunitstayid        
141266             120.40
141448              79.80
141515              70.90
141594             108.30
141786             112.00
...                   ...
3353194             63.05
3353197             71.50
3353199             71.50
3353251            102.00
3353254             83.90

[18825 rows x 1 columns]


In [53]:
#albumin
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('albumin') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
albumin_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
albumin_res = albumin_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
albumin_size = albumin_res.groupby("patientunitstayid").size()
print("albumin max columns:" + str(max(albumin_size)) + "\n" + "albumin avg columns:" + str(sum(albumin_size)/len(albumin_size)) + "\n" + "albumin missing rate:" + str((19615-len(albumin_size))/19615))

length_albumin = int(statistics.median(albumin_size)) + 1

#albumin turn continuous data from vertical to horizontal
albumin_res['cumCount'] = albumin_res.groupby(['patientunitstayid']).cumcount()
albumin = albumin_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
albumin = albumin.iloc[:,:length_albumin]

#albumin let format right
column = []
i = 0
while i < length_albumin:
    column.append("albumin_"+"%s"%str(i))
    i += 1
albumin.columns = column
albumin = albumin.fillna(1000000)
albumin = pd.DataFrame(albumin,dtype = np.float)

#caculate delta

#store the measure offset
albumin_time = albumin_res
#albumin_time turn continuous data from vertical to horizontal
albumin_time['cumCount'] = albumin_time.groupby(['patientunitstayid']).cumcount()
albumin_time = albumin_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
albumin_time = albumin_time.iloc[:,:length_albumin]
albumin_time = albumin_time.fillna(1000000)
column = []
i = 0
while i < length_albumin:
    column.append("albumin_offset_" + "%s"%str(i))
    i += 1
albumin_time.columns = column

i = 1
albumin_delta = albumin
albumin_delta = pd.merge(albumin_delta,albumin_time,on='patientunitstayid',how='left')
while i < length_albumin:
    albumin_delta['albumin_delta_%s'%str(i)] = (albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_%s'%str(i)] - albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_%s'%str(i-1)]) \
    / ((albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_offset_%s'%str(i)] - albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_albumin:
    albumin_delta.drop(columns = ['albumin_%s'%str(i)], inplace=True)
    albumin_delta.drop(columns = ['albumin_offset_%s'%str(i)], inplace=True)
    i += 1
albumin_delta = albumin_delta.fillna(1000000)
albumin_delta = pd.DataFrame(albumin_delta,dtype = np.float)
aki_albumin = albumin
aki_albumin_delta = albumin_delta

print("albumin delta 1 Missing Rate:",1 - albumin_delta[albumin_delta['albumin_delta_1']!=1000000]['albumin_delta_1'].count()/19615)
print(albumin,albumin_delta)


albumin max columns:13
albumin avg columns:1.4089928919415382
albumin missing rate:0.36166199337241905
albumin delta 1 Missing Rate: 0.8101452969666072
                   albumin_0  albumin_1
patientunitstayid                      
141266                   3.6  1000000.0
141304                   3.0  1000000.0
141314                   1.5  1000000.0
141448                   3.7  1000000.0
141470                   2.8  1000000.0
...                      ...        ...
3353144                  2.6  1000000.0
3353190                  4.1  1000000.0
3353194                  3.4        3.1
3353251                  3.3  1000000.0
3353254                  3.9  1000000.0

[12521 rows x 2 columns]                    albumin_delta_1
patientunitstayid                 
141266              1000000.000000
141304              1000000.000000
141314              1000000.000000
141448              1000000.000000
141470              1000000.000000
...                            ...
3353144             10

In [54]:
#creatinine
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('creatinine') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
creatinine_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
creatinine_res = creatinine_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
creatinine_size = creatinine_res.groupby("patientunitstayid").size()
print("creatinine max columns:" + str(max(creatinine_size)) + "\n" + "creatinine avg columns:" + str(sum(creatinine_size)/len(creatinine_size)) + "\n" + "creatinine missing rate:" + str((19615-len(creatinine_size))/19615))

length_creatinine = int(statistics.median(creatinine_size)) + 1

#creatinine turn continuous data from vertical to horizontal
creatinine_res['cumCount'] = creatinine_res.groupby(['patientunitstayid']).cumcount()
creatinine = creatinine_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
creatinine = creatinine.iloc[:,:length_creatinine]

#creatinine let format right
column = []
i = 0
while i < length_creatinine:
    column.append("creatinine_"+"%s"%str(i))
    i += 1
creatinine.columns = column
creatinine = creatinine.fillna(1000000)
creatinine = pd.DataFrame(creatinine,dtype = np.float)

#caculate delta

#store the measure offset
creatinine_time = creatinine_res
#creatinine_time turn continuous data from vertical to horizontal
creatinine_time['cumCount'] = creatinine_time.groupby(['patientunitstayid']).cumcount()
creatinine_time = creatinine_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
creatinine_time = creatinine_time.iloc[:,:length_creatinine]
creatinine_time = creatinine_time.fillna(1000000)
column = []
i = 0
while i < length_creatinine:
    column.append("creatinine_offset_" + "%s"%str(i))
    i += 1
creatinine_time.columns = column

i = 1
creatinine_delta = creatinine
creatinine_delta = pd.merge(creatinine_delta,creatinine_time,on='patientunitstayid',how='left')
while i < length_creatinine:
    creatinine_delta['creatinine_delta_%s'%str(i)] = (creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_%s'%str(i)] - creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_%s'%str(i-1)]) \
    / ((creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_offset_%s'%str(i)] - creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_creatinine:
    creatinine_delta.drop(columns = ['creatinine_%s'%str(i)], inplace=True)
    creatinine_delta.drop(columns = ['creatinine_offset_%s'%str(i)], inplace=True)
    i += 1
creatinine_delta = creatinine_delta.fillna(1000000)
creatinine_delta = pd.DataFrame(creatinine_delta,dtype = np.float)
aki_creatinine = creatinine
aki_creatinine_delta = creatinine_delta

print("creatinine delta 1 Missing Rate:",1 - creatinine_delta[creatinine_delta['creatinine_delta_1']!=1000000]['creatinine_delta_1'].count()/19615)
print(creatinine,creatinine_delta)


creatinine max columns:19
creatinine avg columns:2.0249232436337365
creatinine missing rate:0.15314810094315576
creatinine delta 1 Missing Rate: 0.5202141218455264
                   creatinine_0  creatinine_1  creatinine_2
patientunitstayid                                          
141266                     1.90    1000000.00    1000000.00
141304                     2.34          2.05    1000000.00
141314                     1.67    1000000.00    1000000.00
141448                     1.57    1000000.00    1000000.00
141470                     2.44    1000000.00    1000000.00
...                         ...           ...           ...
3353190                    1.39          1.90    1000000.00
3353194                    1.08          1.21          1.18
3353197                    1.54          1.66    1000000.00
3353251                    1.81    1000000.00    1000000.00
3353254                    2.38    1000000.00    1000000.00

[16611 rows x 3 columns]                    creatinine_

In [55]:
#gfr MDRD fine

#height
cur.execute("SELECT patientunitstayid,admissionheight FROM patient WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) AND admissionheight IS NOT NULL ORDER BY patientunitstayid")
height_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','height'])
height_size = height_res.groupby("patientunitstayid").size()
print("height max columns:" + str(max(height_size)) + "\n" + "height avg columns:" + str(sum(height_size) / len(height_size)) + "\n" + "height missing rate:" + str((19615-len(height_size))/19615))
height_res.fillna(1000000)

#GFR-related
cur.execute("SELECT patientunitstayid,age,gender,ethnicity FROM patient WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
GFR_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','age','gender','ethnicity'])

GFR_res = pd.merge(GFR_res,height_res,on = 'patientunitstayid',how = 'left')
GFR_res = GFR_res.fillna(1000000)
#turn dataframe into list
GFR_res = np.array(GFR_res)
GFR_res = GFR_res.tolist()

for i in GFR_res:
    if i[3] == "African American":
        i[3] = 1
    else:
        i[3] = 0
for i in GFR_res:
    if i[4] > 60:
        i[4] = float(i[4])
    else:
        if i[4] < 2:
            i[4] = 100*float(i[4])
        else:
            i[4] = -1
for i in GFR_res:
    if i[2] == "Female":
        i[2] = 1
    else:
        i[2] = 0
for i in GFR_res:
    if i[1].isdigit() == False:
        i[1] = ''.join([x for x in i[1] if x.isdigit()])
        if i[1] != '':
            i[1] = int(i[1])
        else:
            i[1] = -1
    else:
        i[1] = int(i[1])

GFR_res = pd.DataFrame(GFR_res)
GFR_res.columns = ['patientunitstayid','age','gender','ethnicity','height']
GFR_res.set_index('patientunitstayid', inplace=True)
GFR_res = pd.DataFrame(GFR_res,dtype = np.float)

i = 0
gfr = pd.merge(GFR_res,creatinine,on='patientunitstayid',how='left')
gfr = pd.merge(gfr,BUN,on='patientunitstayid',how='left')
gfr = pd.merge(gfr,albumin,on='patientunitstayid',how='left')
column = ['age','gender','ethnicity','height']
i = 0
while i < length_creatinine:
    column.append("creatinine_"+str("%s"%i))
    i += 1
i = 0
while i < length_BUN:
    column.append("BUN_"+str("%s"%i))
    i += 1
i = 0
while i < length_albumin:
    column.append("albumin_"+str("%s"%i))
    i += 1
gfr.columns = column

#clean the None value
i = 0
gfr.fillna(1000000)
while i < length_creatinine:
    clean_z = gfr["creatinine_"+"%s"%str(i)].fillna(1000000)
    clean_z[clean_z==""] = 1000000
    gfr["creatinine_"+"%s"%str(i)] = clean_z
    i += 1

#gfr MDRD
gfr['age2'] = gfr['age']
gfr['creatinine_02'] = gfr['creatinine_0']
gfr['gender2'] = gfr['gender']
gfr['ethnicity2'] = gfr['ethnicity']
gfr['height2'] = gfr['height']

#age under 17
gfr['gfr_0_C'] = 0.413 * gfr[(gfr['age2'] > 0) & (gfr['age2'] < 17) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['height2'] > 0) & (gfr['height2'] < 300)].height \
/ gfr[(gfr['age2'] > 0) & (gfr['age2'] < 17) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['height2'] > 0) & (gfr['height2'] < 300)].creatinine_0 / 10

#age beyond 17

#men not black
gfr['gfr_0_MNB'] = 186 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 0)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 0) ].age**(-0.203))

#women not black
gfr['gfr_0_WNB'] = 186 * 0.742 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 0)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 0) ].age**(-0.203))

#men black
gfr['gfr_0_MB'] = 186 * 1.210 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 1)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 1) ].age**(-0.203))

#women black
gfr['gfr_0_WB'] = 186 * 1.210 * 0.742 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 1)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 1) ].age**(-0.203))

gfr = gfr.fillna(0)
#gfr.reset_index(inplace = True)
print()
gfr['gfr'] = gfr['gfr_0_WB'] + gfr['gfr_0_MB'] + gfr['gfr_0_WNB'] + gfr['gfr_0_MNB'] + gfr['gfr_0_C']
gfr[gfr['gfr'] == 0] = 1000000
gfr = pd.DataFrame(gfr,dtype = np.float)

gfr.drop(columns = ['age2','gender2','ethnicity2','height2','creatinine_02'], inplace=True)
gfr.drop(columns = ['gfr_0_WB','gfr_0_MB','gfr_0_WNB','gfr_0_MNB','gfr_0_C'], inplace=True)

for i in gfr[gfr['gfr'] > 180].index.values:
    gfr.loc[i]['gfr'] = 1000000

print("gfr Missing Rate :",1 - gfr[gfr['gfr'] != 1000000]['gfr'].count()/19615)
# print(gfr[(gfr['gfr'] != 1000000) & (gfr['gfr'] >200)]['gfr'])

gfr['age1'] = gfr['age']
gfr['creatinine_01'] = gfr['creatinine_0']
gfr['gender1'] = gfr['gender']
gfr['ethnicity1'] = gfr['ethnicity']

#gfr CKD_EPI

#men not black
gfr['gfr_0_MNB'] = 141 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.9) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.9)**(-0.411)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_MNB1'] = 141 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.9) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.9)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

#women not black
gfr['gfr_0_WNB'] = 144 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.7) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.7)**(-0.329)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_WNB1'] = 144 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.7) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.7)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0) ].age))

#men black
gfr['gfr_0_MB'] = 163 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.9) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.9)**(-0.411)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_MB1'] = 163 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.9) & (gfr['creatinine_01'] <= 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.9)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

#women black
gfr['gfr_0_WB'] = 166 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.7) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.7)**(-0.329)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_WB1'] = 166 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.7) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.7)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr = gfr.fillna(0)
#gfr.reset_index(inplace = True)
gfr['gfr_CKI'] = gfr['gfr_0_WB'] + gfr['gfr_0_MB'] + gfr['gfr_0_WNB'] + gfr['gfr_0_MNB'] + gfr['gfr_0_WB1'] + gfr['gfr_0_MB1'] + gfr['gfr_0_WNB1'] + gfr['gfr_0_MNB1']
gfr[gfr['gfr_CKI'] == 0] = 1000000
gfr = pd.DataFrame(gfr,dtype = np.float)

gfr.drop(columns = ['age1','gender1','ethnicity1','creatinine_01'], inplace=True)
gfr.drop(columns = ['gfr_0_WB','gfr_0_MB','gfr_0_WNB','gfr_0_MNB','gfr_0_WB1','gfr_0_MB1','gfr_0_WNB1','gfr_0_MNB1'], inplace=True)

for i in gfr[gfr['gfr_CKI'] > 180].index.values:
    gfr.loc[i]['gfr_CKI'] = 1000000

print("gfr_CKI Missing Rate :",1 - gfr[gfr['gfr_CKI'] != 1000000]['gfr_CKI'].count()/19615)


#gfr MDRD fine

#age under 17
gfr['gfr_0_C'] = 0.412 * gfr[(gfr['age'] > 0) & (gfr['age'] < 17) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['height'] > 0) & (gfr['height'] < 300)].height \
/ gfr[(gfr['age'] > 0) & (gfr['age'] < 17) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['height'] > 0) & (gfr['height'] < 300)].creatinine_0  / 10

#age beyond 17

#men not black
gfr['gfr_0_MNB'] = (170 * (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318)))

gfr['gfr_0_WNB'] = (170 * 0.762 *(gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318)))

#men black
gfr['gfr_0_MB'] = 170 * 1.18 * (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318))

#women black
gfr['gfr_0_WB'] = 170 * 1.18 * 0.762 *(gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318))

gfr = gfr.fillna(0)
#gfr.reset_index(inplace = True)
gfr['gfr_fine'] = gfr['gfr_0_WB'] + gfr['gfr_0_MB'] + gfr['gfr_0_WNB'] + gfr['gfr_0_MNB'] + gfr['gfr_0_C']
gfr[gfr['gfr_fine'] == 0] = 1000000
gfr = pd.DataFrame(gfr,dtype = np.float)

gfr.drop(columns = ['age','gender','ethnicity','height','creatinine_0','creatinine_1','BUN_0','BUN_1','albumin_0','albumin_1'], inplace=True)
gfr.drop(columns = ['gfr_0_WB','gfr_0_MB','gfr_0_WNB','gfr_0_MNB','gfr_0_C'], inplace=True)
aki_gfr = gfr

for i in gfr[gfr['gfr_fine'] > 180].index.values:
    gfr.loc[i]['gfr_fine'] = 1000000

print("gfr_fine Missing Rate :",1 - gfr[gfr['gfr_fine'] != 1000000]['gfr_fine'].count()/19615)
print(gfr)


height max columns:1
height avg columns:1.0
height missing rate:0.008972724955391281


/Users/rocky/opt/anaconda3/envs/py3_7/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)



gfr Missing Rate : 0.26505225592658677
gfr_CKI Missing Rate : 0.3444302829467244
gfr_fine Missing Rate : 0.5884272240632169
                   creatinine_2             gfr         gfr_CKI  \
patientunitstayid                                                 
141266               1000000.00       37.117332       34.212148   
141304               1000000.00       29.436281       27.162124   
141314               1000000.00       47.522141       48.679967   
141448               1000000.00       47.361269       45.578159   
141470               1000000.00       21.621863       21.173280   
...                         ...             ...             ...   
3353194                    1.18       56.847202       59.578337   
3353197              1000000.00  1000000.000000  1000000.000000   
3353199              1000000.00  1000000.000000  1000000.000000   
3353251              1000000.00  1000000.000000  1000000.000000   
3353254              1000000.00       28.023390       24.632353   

   

In [56]:

print(max(gfr[gfr['gfr_CKI'] != 1000000]['gfr_CKI']))


145.4734749078661


In [57]:
#condition-related
cur.execute("SELECT patientunitstayid,cirrhosis,gender,hepaticFailure,ventday1,diabetes FROM apachepredvar WHERE patientunitstayid IN (SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
condition = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','cirrhosis','gender','hepaticFailure','ventday1','diabetes'])
condition = condition.fillna(1000000)
condition.set_index('patientunitstayid',inplace=True)
aki_condition = condition

print(condition)
print("cirrhosis Missing Rate:",1 - condition[condition['cirrhosis']!=1000000]['cirrhosis'].count()/19615)
print("gender Missing Rate:",1 - condition[condition['gender']!=1000000]['gender'].count()/19615)
print("hepaticFailure Missing Rate:",1 - condition[condition['hepaticFailure']!=1000000]['hepaticFailure'].count()/19615)
print("ventday1 Missing Rate:",1 - condition[condition['ventday1']!=1000000]['ventday1'].count()/19615)
print("diabetes Missing Rate:",1 - condition[condition['diabetes']!=1000000]['diabetes'].count()/19615)


                   cirrhosis  gender  hepaticFailure  ventday1  diabetes
patientunitstayid                                                       
141266                     0       0               0         0         1
141304                     0       0               0         1         0
141314                     0       0               0         1         0
141448                     0       0               0         0         0
141470                     0       1               0         0         0
...                      ...     ...             ...       ...       ...
3353190                    0       0               0         0         0
3353194                    0       1               0         1         1
3353197                    0       1               0         0         0
3353251                    0       0               0         1         1
3353254                    0       0               0         0         0

[18563 rows x 5 columns]
cirrhosis Missing Rate: 0

In [58]:
#glucose
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('glucose') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
glucose_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
glucose_res = glucose_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
glucose_size = glucose_res.groupby("patientunitstayid").size()
print("glucose max columns:" + str(max(glucose_size)) + "\n" + "glucose avg columns:" + str(sum(glucose_size)/len(glucose_size)) + "\n" + "glucose missing rate:" + str((19615-len(glucose_size))/19615))

length_glucose = int(statistics.median(glucose_size)) + 1

#glucose turn continuous data from vertical to horizontal
glucose_res['cumCount'] = glucose_res.groupby(['patientunitstayid']).cumcount()
glucose = glucose_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
glucose = glucose.iloc[:,:length_glucose]

#glucose let format right
column = []
i = 0
while i < length_glucose:
    column.append("glucose_"+"%s"%str(i))
    i += 1
glucose.columns = column
glucose = glucose.fillna(1000000)
glucose = pd.DataFrame(glucose,dtype = np.float)

#caculate delta

#store the measure offset
glucose_time = glucose_res
#glucose_time turn continuous data from vertical to horizontal
glucose_time['cumCount'] = glucose_time.groupby(['patientunitstayid']).cumcount()
glucose_time = glucose_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
glucose_time = glucose_time.iloc[:,:length_glucose]
glucose_time = glucose_time.fillna(1000000)
column = []
i = 0
while i < length_glucose:
    column.append("glucose_offset_" + "%s"%str(i))
    i += 1
glucose_time.columns = column

i = 1
glucose_delta = glucose
glucose_delta = pd.merge(glucose_delta,glucose_time,on='patientunitstayid',how='left')
while i < length_glucose:
    glucose_delta['glucose_delta_%s'%str(i)] = (glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_%s'%str(i)] - glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_%s'%str(i-1)]) \
    / ((glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_offset_%s'%str(i)] - glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_glucose:
    glucose_delta.drop(columns = ['glucose_%s'%str(i)], inplace=True)
    glucose_delta.drop(columns = ['glucose_offset_%s'%str(i)], inplace=True)
    i += 1
glucose_delta = glucose_delta.fillna(1000000)
glucose_delta = pd.DataFrame(glucose_delta,dtype = np.float)
for i in glucose[(glucose['glucose_0'] > 2000) & (glucose['glucose_0'] != 1000000)].index.values:
    glucose.loc[i]['glucose_0'] = 1000000
for i in glucose[(glucose['glucose_1'] > 2000) & (glucose['glucose_1'] != 1000000)].index.values:
    glucose.loc[i]['glucose_1'] = 1000000
aki_glucose = glucose
aki_glucose_delta = glucose_delta

print(glucose,glucose_delta)
print("glucose delta 1 Missing Rate:",1 - glucose_delta[glucose_delta['glucose_delta_1']!=1000000]['glucose_delta_1'].count()/19615)


glucose max columns:30
glucose avg columns:2.125007578053956
glucose missing rate:0.15906194239102728
                   glucose_0  glucose_1  glucose_2
patientunitstayid                                 
141266                 423.0  1000000.0  1000000.0
141304                 173.0      129.0  1000000.0
141314                 153.0  1000000.0  1000000.0
141448                 232.0  1000000.0  1000000.0
141470                 153.0  1000000.0  1000000.0
...                      ...        ...        ...
3353190                 84.0       69.0  1000000.0
3353194                254.0      292.0      204.0
3353197                 85.0      264.0  1000000.0
3353251                305.0  1000000.0  1000000.0
3353254                181.0  1000000.0  1000000.0

[16495 rows x 3 columns]                    glucose_delta_1  glucose_delta_2
patientunitstayid                                  
141266              1000000.000000   1000000.000000
141304                  -12.000000   1000000.000000
1

In [59]:
sql="SELECT T.patientunitstayid,T.treatmentoffset,T.treatmentstring FROM (SELECT MAX(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,treatment T WHERE TB.patientunitstayid=T.patientunitstayid AND treatmentstring LIKE '%renal|dialysis%' AND T.treatmentoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,treatmentoffset"
cur.execute(sql)
dialysis_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','treatmentoffset','treatmentstring'])
dialysis_res = dialysis_res.groupby(['patientunitstayid']).agg({'treatmentoffset':np.min}).reset_index()
dialysis_res['treatmentoffset'] = 1
dialysis_res=dialysis_res.rename(columns={'treatmentoffset':'IsDialysis'})
dialysis_res=pd.merge(akiDiagdf['patientunitstayid'],dialysis_res,on='patientunitstayid',how='left')
dialysis_res=dialysis_res.fillna(0)
dialysis = dialysis_res
dialysis.set_index('patientunitstayid',inplace=True)
aki_dialysis = dialysis
print(dialysis)


                   IsDialysis
patientunitstayid            
141266                    0.0
141304                    0.0
141314                    0.0
141448                    0.0
141470                    0.0
...                       ...
3353194                   0.0
3353197                   0.0
3353199                   0.0
3353251                   0.0
3353254                   0.0

[19615 rows x 1 columns]


In [60]:
#show how to concat two dataframe,use for loop to connect all dataframe(must start with age_res,since its missing rate is 0)
AKIPT = pd.DataFrame()
AKIPT = pd.merge(aki_age,aki_nettotal,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_dialysis,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_potassium,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_sodium,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_Hct,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_BUN,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_pH,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_OI,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_TP,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_TP_sum,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_urine,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_diastolicBP,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_systolicBP,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_meanBP,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_creatinine,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_glucose,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_albumin,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_gfr,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_pulseP,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_MSI,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_weight,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_condition,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_creatinine_delta,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_urine_delta,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_diastolicBP_delta,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_systolicBP_delta,on='patientunitstayid',how='left')
# AKIPT = pd.merge(AKIPT,aki_meanBP_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_potassium_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_sodium_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_Hct_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_BUN_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_pH_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_TP_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_glucose_delta,on='patientunitstayid',how='left')
AKIPT = pd.merge(AKIPT,aki_albumin_delta,on='patientunitstayid',how='left')
AKIPT['aki'] = 1
AKIPT=AKIPT.fillna(1000000)
AKIPT = AKIPT.loc[AKIPT['IsDialysis'] != 1]
AKIPT.drop('IsDialysis',axis = 1,inplace = True)
print(AKIPT)
AKIPT.to_csv('AKIPT.csv', encoding='utf-8')

                    age    nettotal  potassium_0  potassium_1   sodium_0  \
patientunitstayid                                                          
141266             73.0       63.22          5.2    1000000.0      135.0   
141304             70.0     1253.00          4.8          5.8      141.0   
141314             45.0      150.00          8.0    1000000.0      158.0   
141448             65.0        0.00          2.8    1000000.0      138.0   
141470             58.0        0.00          4.3    1000000.0      131.0   
...                 ...         ...          ...          ...        ...   
3353194            51.0     -468.00          4.7          3.2      137.0   
3353197            66.0  1000000.00          4.3          4.2      134.0   
3353199            66.0       17.44    1000000.0    1000000.0  1000000.0   
3353251            73.0       60.13          3.9          3.9      136.0   
3353254            81.0      500.00          5.0    1000000.0      141.0   

           

In [132]:
#get NOAKI patient's ID and diagnosisoffset
cur.execute("SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid")
akiDiagdf = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','diagnosisoffset'])


#net fluid balance
cur = conn.cursor()
cur.execute("SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.nettotal FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY IO.patientunitstayid,IO.intakeoutputoffset")
IO_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','intakeoutputoffset','nettotal'])
IO_res=IO_res.groupby(['patientunitstayid','intakeoutputoffset']).agg({'nettotal':np.min}).reset_index()
IO_res=IO_res.groupby(['patientunitstayid']).agg({'nettotal':np.sum}).reset_index()  
nettotal = IO_res
nettotal.columns = ['patientunitstayid','nettotal']
nettotal.set_index('patientunitstayid',inplace=True)
nettotal=nettotal.fillna(1000000)
nettotal = pd.DataFrame(nettotal,dtype = np.float)
for i in nettotal[(nettotal['nettotal'] != 1000000) & (nettotal['nettotal'] > 3000)].index.values:
    nettotal.loc[i]['nettotal'] = 1000000

print("nettotal Missing Rate:",1 - nettotal[nettotal['nettotal']!=1000000]['nettotal'].count()/181244)
print(nettotal)


nettotal Missing Rate: 0.4565006289863388
                   nettotal
patientunitstayid          
141168                 0.00
141203                 0.00
141227              1830.00
141288              2351.00
141296               620.00
...                     ...
3353216            -2138.56
3353226                1.41
3353235                0.00
3353237            -4436.50
3353263               16.67

[100021 rows x 1 columns]


In [133]:
#potassium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('potassium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
potassium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','labresult'])
potassium_res=potassium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
potassium_size=potassium_res.groupby("patientunitstayid").size()
print("potassium max columns:" + str(max(potassium_size)) + "\n" + "potassium avg columns:" + str(sum(potassium_size)/len(potassium_size)) + "\n" + "potassium missing rate:" + str((181244-len(potassium_size))/181244))
#potassium_res.set_index(['patientunitstayid','labresultoffset']).index.is_unique

#potassium turn continuous data from vertical to horizontal
potassium_res['cumCount'] = potassium_res.groupby(['patientunitstayid']).cumcount()
potassium = potassium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
potassium = potassium.iloc[:,:length_potassium]

#potassium let format right
potassium = pd.DataFrame(potassium)
potassium = potassium.fillna(1000000)
potassium = pd.DataFrame(potassium,dtype = np.float)
column = []
i = 0
while i < length_potassium:
    column.append("potassium_" + "%s"%str(i))
    i += 1
potassium.columns = column

#caculate delta

#store the measure offset
potassium_time = potassium_res
#potassium_time turn continuous data from vertical to horizontal
potassium_time['cumCount'] = potassium_time.groupby(['patientunitstayid']).cumcount()
potassium_time = potassium_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
potassium_time = potassium_time.iloc[:,:length_potassium]
potassium_time = potassium_time.fillna(1000000)
column = []
i = 0
while i < length_potassium:
    column.append("potassium_offset_" + "%s"%str(i))
    i += 1
potassium_time.columns = column

i = 1
potassium_delta = potassium
potassium_delta = pd.merge(potassium_delta,potassium_time,on='patientunitstayid',how='left')
while i < length_potassium:
    potassium_delta['potassium_delta_%s'%str(i)] = (potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_%s'%str(i)] - potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_%s'%str(i-1)]) \
    / ((potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_offset_%s'%str(i)] - potassium_delta[potassium_delta['potassium_%s'%str(i)] < 1000000]['potassium_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_potassium:
    potassium_delta.drop(columns = ['potassium_%s'%str(i)], inplace=True)
    potassium_delta.drop(columns = ['potassium_offset_%s'%str(i)], inplace=True)
    i += 1
potassium_delta = potassium_delta.fillna(1000000)
potassium_delta = pd.DataFrame(potassium_delta,dtype = np.float)
for i in potassium[(potassium['potassium_0'] != 1000000) & (potassium['potassium_0'] > 10)].index.values:
    potassium.loc[i]['potassium_0'] = 1000000
for i in potassium[(potassium['potassium_1'] != 1000000) & (potassium['potassium_1'] > 10)].index.values:
    potassium.loc[i]['potassium_1'] = 1000000

print("potassium delta Missing Rate:",1 - potassium_delta[potassium_delta['potassium_delta_1']!=1000000]['potassium_delta_1'].count()/181244)
print(potassium,potassium_delta)


potassium max columns:31
potassium avg columns:1.5399824927128174
potassium missing rate:0.38860872635783805
potassium delta Missing Rate: 0.8259969985213304
                   potassium_0  potassium_1
patientunitstayid                          
141203                     3.6          4.2
141227                     4.2          4.1
141229                     4.0    1000000.0
141288                     3.7    1000000.0
141296                     5.4    1000000.0
...                        ...          ...
3353213                    3.8    1000000.0
3353216                    5.1    1000000.0
3353226                    4.8    1000000.0
3353235                    4.1    1000000.0
3353263                    4.1    1000000.0

[110811 rows x 2 columns]                    potassium_delta_1
patientunitstayid                   
141203                      0.075949
141227                     -0.052174
141229                1000000.000000
141288                1000000.000000
141296               

In [134]:
#sodium
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('sodium') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
sodium_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','labresult'])
sodium_res=sodium_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
sodium_size=sodium_res.groupby("patientunitstayid").size()
print("sodium max columns:" + str(max(sodium_size)) + "\n" + "sodium avg columns:" + str(sum(sodium_size)/len(sodium_size)) + "\n" + "sodium missing rate:" + str((181244-len(sodium_size))/181244))

#sodium turn continuous data from vertical to horizontal
sodium_res['cumCount'] = sodium_res.groupby(['patientunitstayid']).cumcount()
sodium = sodium_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
sodium = sodium.iloc[:,:length_sodium]

#sodium let format right
column = []
i = 0
while i < length_sodium:
    column.append("sodium_" + "%s"%str(i))
    i += 1
sodium.columns = column
sodium = sodium.fillna(1000000)
sodium = pd.DataFrame(sodium,dtype = np.float)

#caculate delta

#store the measure offset
sodium_time = sodium_res
#sodium_time turn continuous data from vertical to horizontal
sodium_time['cumCount'] = sodium_time.groupby(['patientunitstayid']).cumcount()
sodium_time = sodium_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
sodium_time = sodium_time.iloc[:,:length_sodium]
sodium_time = sodium_time.fillna(1000000)
column = []
i = 0
while i < length_sodium:
    column.append("sodium_offset_" + "%s"%str(i))
    i += 1
sodium_time.columns = column

i = 1
sodium_delta = sodium
sodium_delta = pd.merge(sodium_delta,sodium_time,on='patientunitstayid',how='left')
while i < length_sodium:
    sodium_delta['sodium_delta_%s'%str(i)] = (sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_%s'%str(i)] - sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_%s'%str(i-1)]) \
    / ((sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_offset_%s'%str(i)] - sodium_delta[sodium_delta['sodium_%s'%str(i)] < 1000000]['sodium_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_sodium:
    sodium_delta.drop(columns = ['sodium_%s'%str(i)], inplace=True)
    sodium_delta.drop(columns = ['sodium_offset_%s'%str(i)], inplace=True)
    i += 1
sodium_delta = sodium_delta.fillna(1000000)
sodium_delta = pd.DataFrame(sodium_delta,dtype = np.float)
for i in sodium[(sodium['sodium_0'] != 1000000) & (sodium['sodium_0'] > 180)].index.values:
    sodium.loc[i]['sodium_0'] = 1000000
for i in sodium[(sodium['sodium_1'] != 1000000) & (sodium['sodium_1'] > 180)].index.values:
    sodium.loc[i]['sodium_1'] = 1000000
for i in sodium[(sodium['sodium_0'] != 1000000) & (sodium['sodium_0'] < 100)].index.values:
    sodium.loc[i]['sodium_0'] = 1000000
for i in sodium[(sodium['sodium_1'] != 1000000) & (sodium['sodium_1'] < 100)].index.values:
    sodium.loc[i]['sodium_1'] = 1000000

print("sodium delta Missing Rate:",1 - sodium_delta[sodium_delta['sodium_delta_1']!=1000000]['sodium_delta_1'].count()/181244)
print(sodium,sodium_delta)


sodium max columns:31
sodium avg columns:1.5125738840337963
sodium missing rate:0.39138399064244883
sodium delta Missing Rate: 0.8343779656154134
                   sodium_0   sodium_1
patientunitstayid                     
141203                144.0      145.0
141227                133.0      132.0
141229                138.0  1000000.0
141288                142.0  1000000.0
141296                138.0  1000000.0
...                     ...        ...
3353213               143.0  1000000.0
3353216               143.0  1000000.0
3353226               136.0  1000000.0
3353235               137.0  1000000.0
3353263               135.0  1000000.0

[110308 rows x 2 columns]                    sodium_delta_1
patientunitstayid                
141203                   0.126582
141227                  -0.521739
141229             1000000.000000
141288             1000000.000000
141296             1000000.000000
...                           ...
3353213            1000000.000000
3353216       

In [135]:
#Hct
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('Hct') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
Hct_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','labresult'])
Hct_res=Hct_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
Hct_size=Hct_res.groupby("patientunitstayid").size()
print("Hct max columns:" + str(max(Hct_size)) + "\n" + "Hct avg columns:" + str(sum(Hct_size)/len(Hct_size)) + "\n" + "Hct missing rate:" + str((181244-len(Hct_size))/181244))

#Hct turn continuous data from vertical to horizontal
Hct_res['cumCount'] = Hct_res.groupby(['patientunitstayid']).cumcount()
Hct = Hct_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
Hct = Hct.iloc[:,:length_Hct]

#Hct let format right
column = []
i = 0
while i < length_Hct:
    column.append("Hct_" + "%s"%str(i))
    i += 1
Hct.columns = column
Hct = Hct.fillna(1000000)
Hct = pd.DataFrame(Hct,dtype = np.float)

#caculate delta

#store the measure offset
Hct_time = Hct_res
#Hct_time turn continuous data from vertical to horizontal
Hct_time['cumCount'] = Hct_time.groupby(['patientunitstayid']).cumcount()
Hct_time = Hct_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
Hct_time = Hct_time.iloc[:,:length_Hct]
Hct_time = Hct_time.fillna(1000000)
column = []
i = 0
while i < length_Hct:
    column.append("Hct_offset_" + "%s"%str(i))
    i += 1
Hct_time.columns = column

i = 1
Hct_delta = Hct
Hct_delta = pd.merge(Hct_delta,Hct_time,on='patientunitstayid',how='left')
while i < length_Hct:
    Hct_delta['Hct_delta_%s'%str(i)] = (Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_%s'%str(i)] - Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_%s'%str(i-1)]) \
    / ((Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_offset_%s'%str(i)] - Hct_delta[Hct_delta['Hct_%s'%str(i)] < 1000000]['Hct_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_Hct:
    Hct_delta.drop(columns = ['Hct_%s'%str(i)], inplace=True)
    Hct_delta.drop(columns = ['Hct_offset_%s'%str(i)], inplace=True)
    i += 1
Hct_delta = Hct_delta.fillna(1000000)
Hct_delta = pd.DataFrame(Hct_delta,dtype = np.float)
for i in Hct[(Hct['Hct_0'] != 1000000) & (Hct['Hct_0'] > 70)].index.values:
    Hct.loc[i]['Hct_0'] = 1000000
for i in Hct[(Hct['Hct_1'] != 1000000) & (Hct['Hct_1'] > 70)].index.values:
    Hct.loc[i]['Hct_1'] = 1000000
for i in Hct[(Hct['Hct_0'] != 1000000) & (Hct['Hct_0'] < 15)].index.values:
    Hct.loc[i]['Hct_0'] = 1000000
for i in Hct[(Hct['Hct_1'] != 1000000) & (Hct['Hct_1'] < 15)].index.values:
    Hct.loc[i]['Hct_1'] = 1000000

print("Hct delta Missing Rate:",1 - Hct_delta[Hct_delta['Hct_delta_1']!=1000000]['Hct_delta_1'].count()/181244)
print(Hct,Hct_delta)


Hct max columns:32
Hct avg columns:1.4486312746339143
Hct missing rate:0.38546379466354747
Hct delta Missing Rate: 0.8518350952307386
                   Hct_0      Hct_1
patientunitstayid                  
141203              35.0       36.9
141227              27.0       28.6
141229              38.2  1000000.0
141288              43.7  1000000.0
141296              30.0  1000000.0
...                  ...        ...
3353213             51.0  1000000.0
3353216             26.0  1000000.0
3353226             28.0  1000000.0
3353235             49.0  1000000.0
3353263             44.0  1000000.0

[111381 rows x 2 columns]                       Hct_delta_1
patientunitstayid                
141203                   0.240506
141227                   0.834783
141229             1000000.000000
141288             1000000.000000
141296             1000000.000000
...                           ...
3353213            1000000.000000
3353216            1000000.000000
3353226            1000000.0000

In [136]:
#BUN
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('BUN') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
BUN_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','labresult'])
BUN_res=BUN_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
BUN_size=BUN_res.groupby("patientunitstayid").size()
print("BUN max columns:" + str(max(BUN_size)) + "\n" + "BUN avg columns:" + str(sum(BUN_size)/len(BUN_size)) + "\n" + "BUN missing rate:" + str((181244-len(BUN_size))/181244))

#BUN turn continuous data from vertical to horizontal
BUN_res['cumCount'] = BUN_res.groupby(['patientunitstayid']).cumcount()
BUN = BUN_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
BUN = BUN.iloc[:,:length_BUN]

#BUN let format right
column = []
i = 0
while i < length_BUN:
    column.append("BUN_" + "%s"%str(i))
    i += 1
BUN.columns = column
BUN = BUN.fillna(1000000)
BUN = pd.DataFrame(BUN,dtype = np.float)

#caculate delta

#store the measure offset
BUN_time = BUN_res
#BUN_time turn continuous data from vertical to horizontal
BUN_time['cumCount'] = BUN_time.groupby(['patientunitstayid']).cumcount()
BUN_time = BUN_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
BUN_time = BUN_time.iloc[:,:length_BUN]
BUN_time = BUN_time.fillna(1000000)
column = []
i = 0
while i < length_BUN:
    column.append("BUN_offset_" + "%s"%str(i))
    i += 1
BUN_time.columns = column

i = 1
BUN_delta = BUN
BUN_delta = pd.merge(BUN_delta,BUN_time,on='patientunitstayid',how='left')
while i < length_BUN:
    BUN_delta['BUN_delta_%s'%str(i)] = (BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_%s'%str(i)] - BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_%s'%str(i-1)]) \
    / ((BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_offset_%s'%str(i)] - BUN_delta[BUN_delta['BUN_%s'%str(i)] < 1000000]['BUN_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_BUN:
    BUN_delta.drop(columns = ['BUN_%s'%str(i)], inplace=True)
    BUN_delta.drop(columns = ['BUN_offset_%s'%str(i)], inplace=True)
    i += 1
BUN_delta = BUN_delta.fillna(1000000)
BUN_delta = pd.DataFrame(BUN_delta,dtype = np.float)
for i in BUN[(BUN['BUN_0'] != 1000000) & (BUN['BUN_0'] > 70)].index.values:
    BUN.loc[i]['BUN_0'] = 1000000
for i in BUN[(BUN['BUN_1'] != 1000000) & (BUN['BUN_1'] > 70)].index.values:
    BUN.loc[i]['BUN_0'] = 1000000
for i in BUN[(BUN['BUN_0'] != 1000000) & (BUN['BUN_0'] < 1)].index.values:
    BUN.loc[i]['BUN_0'] = 1000000
for i in BUN[(BUN['BUN_1'] != 1000000) & (BUN['BUN_1'] < 1)].index.values:
    BUN.loc[i]['BUN_1'] = 1000000

print("BUN delta Missing Rate:",1 - BUN_delta[BUN_delta['BUN_delta_1']!=1000000]['BUN_delta_1'].count()/181244)
print(BUN,BUN_delta)


BUN max columns:11
BUN avg columns:1.2355941845175369
BUN missing rate:0.4083611043675929
BUN delta Missing Rate: 0.8824457637218336
                   BUN_0      BUN_1
patientunitstayid                  
141203               9.0        9.0
141227              26.0       26.0
141229              25.0  1000000.0
141288              17.0  1000000.0
141296              49.0  1000000.0
...                  ...        ...
3353213              9.0  1000000.0
3353216              7.0  1000000.0
3353226             42.0  1000000.0
3353235             16.0  1000000.0
3353263             13.0  1000000.0

[107231 rows x 2 columns]                    BUN_delta_1
patientunitstayid             
141203                     0.0
141227                     0.0
141229               1000000.0
141288               1000000.0
141296               1000000.0
...                        ...
3353213              1000000.0
3353216              1000000.0
3353226              1000000.0
3353235              1000000.0


In [137]:
#pH
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('pH') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
pH_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','labresult'])
pH_res=pH_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
pH_size=pH_res.groupby("patientunitstayid").size()
print("pH max columns:" + str(max(pH_size)) + "\n" + "pH avg columns:" + str(sum(pH_size)/len(pH_size)) + "\n" + "pH missing rate:" + str((181244-len(pH_size))/181244))

#pH turn continuous data from vertical to horizontal
pH_res['cumCount'] = pH_res.groupby(['patientunitstayid']).cumcount()
pH = pH_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
pH = pH.iloc[:,:length_pH]

#pH let format right
column = []
i = 0
while i < length_pH:
    column.append("pH_" + "%s"%str(i))
    i += 1
pH.columns = column
pH = pH.fillna(1000000)
pH = pd.DataFrame(pH,dtype = np.float)

#caculate delta

#store the measure offset
pH_time = pH_res
#pH_time turn continuous data from vertical to horizontal
pH_time['cumCount'] = pH_time.groupby(['patientunitstayid']).cumcount()
pH_time = pH_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
pH_time = pH_time.iloc[:,:length_pH]
pH_time = pH_time.fillna(1000000)
column = []
i = 0
while i < length_pH:
    column.append("pH_offset_" + "%s"%str(i))
    i += 1
pH_time.columns = column

i = 1
pH_delta = pH
pH_delta = pd.merge(pH_delta,pH_time,on='patientunitstayid',how='left')
while i < length_pH:
    pH_delta['pH_delta_%s'%str(i)] = (pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_%s'%str(i)] - pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_%s'%str(i-1)]) \
    / ((pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_offset_%s'%str(i)] - pH_delta[pH_delta['pH_%s'%str(i)] < 1000000]['pH_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_pH:
    pH_delta.drop(columns = ['pH_%s'%str(i)], inplace=True)
    pH_delta.drop(columns = ['pH_offset_%s'%str(i)], inplace=True)
    i += 1
pH_delta = pH_delta.fillna(1000000)
pH_delta = pd.DataFrame(pH_delta,dtype = np.float)
for i in pH[(pH['pH_0'] != 1000000) & (pH['pH_0'] > 7.8)].index.values:
    pH.loc[i]['pH_0'] = 1000000
for i in pH[(pH['pH_1'] != 1000000) & (pH['pH_1'] > 7.8)].index.values:
    pH.loc[i]['pH_1'] = 1000000
for i in pH[(pH['pH_0'] != 1000000) & (pH['pH_0'] < 6.8)].index.values:
    pH.loc[i]['pH_0'] = 1000000
for i in pH[(pH['pH_1'] != 1000000) & (pH['pH_1'] < 6.8)].index.values:
    pH.loc[i]['pH_1'] = 1000000

print("pH delta Missing Rate:",1 - pH_delta[pH_delta['pH_delta_1']!=1000000]['pH_delta_1'].count()/181244)
print(pH,pH_delta)


pH max columns:31
pH avg columns:1.990486701247103
pH missing rate:0.7500331045441504
pH delta Missing Rate: 0.9074176248593057
                    pH_0         pH_1
patientunitstayid                    
141203             7.450  1000000.000
141227             7.370  1000000.000
141296             7.540  1000000.000
141297             7.420  1000000.000
141313             7.140        7.200
...                  ...          ...
3353147            7.308        7.393
3353201            7.566  1000000.000
3353213            7.200        7.294
3353216            7.185        7.293
3353226            7.180        7.283

[45305 rows x 2 columns]                        pH_delta_1
patientunitstayid                
141203             1000000.000000
141227             1000000.000000
141296             1000000.000000
141297             1000000.000000
141313                   0.046753
...                           ...
3353147                  0.030357
3353201            1000000.000000
3353213     

In [138]:
#paO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('paO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
paO2_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','paO2_labresult'])
paO2_res=paO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'paO2_labresult':np.min}).reset_index()
paO2_size=paO2_res.groupby("patientunitstayid").size()
print("paO2 max columns:" + str(max(paO2_size)) + "\n" + "paO2 avg columns:" + str(sum(paO2_size)/len(paO2_size)) + "\n" + "paO2 missing rate:" + str((181244-len(paO2_size))/181244))

#FiO2
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('FiO2') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
FiO2_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','FiO2_labresult'])
FiO2_res=FiO2_res.groupby(['patientunitstayid','labresultoffset']).agg({'FiO2_labresult':np.min}).reset_index()
FiO2_size=FiO2_res.groupby("patientunitstayid").size()
print("FiO2 max columns:" + str(max(FiO2_size)) + "\n" + "FiO2 avg columns:" + str(sum(FiO2_size)/len(FiO2_size)) + "\n" + "FiO2 missing rate:" + str((181244-len(FiO2_size))/181244))

#pick FiO2 and paO2 at the same time,can use this dataframe to calculate ratio
OI=pd.DataFrame()
OI=pd.merge(paO2_res,FiO2_res,on=['patientunitstayid','labresultoffset'],how='inner')
OI_size=OI.groupby("patientunitstayid").size()
print("paO2/FiO2 max columns:" + str(max(OI_size)) + "\n" + "paO2/FiO2 avg columns:" + str(sum(OI_size)/len(OI_size)) + "\n" + "paO2/FiO2 missing rate:" + str((181244-len(OI_size))/181244))

i = 0
clean_z = OI['FiO2_labresult'].fillna(1000000)
clean_z[clean_z == ''] = 1000000
OI['FiO2_labresult'] = clean_z
clean_z = OI['FiO2_labresult'].fillna(1000000)
clean_z[clean_z ==  0] = 1000000
OI['FiO2_labresult'] = clean_z
OI['OI'] = OI['paO2_labresult'] / OI['FiO2_labresult']
OI.drop(columns = ['paO2_labresult','FiO2_labresult'], inplace = True)

#OI turn continuous data from vertical to horizontal
OI['cumCount'] = OI.groupby(['patientunitstayid']).cumcount()
OI = OI.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'OI')
OI = OI.iloc[:,:length_OI]

#OI let format right
column = []
i = 0
while i < length_OI:
    column.append("OI_"+"%s"%str(i))
    i += 1
OI.columns = column
OI = OI.fillna(1000000)
OI = pd.DataFrame(OI,dtype = np.float)

print("OI 0 Missing Rate:",1 - OI[OI['OI_0']!=1000000]['OI_0'].count()/181244)
print(OI)


paO2 max columns:31
paO2 avg columns:1.9753754915232387
paO2 missing rate:0.7432246032972126
FiO2 max columns:23
FiO2 avg columns:1.470459320846795
FiO2 missing rate:0.7557932952263248
paO2/FiO2 max columns:23
paO2/FiO2 avg columns:1.4881642936661392
paO2/FiO2 missing rate:0.8116682483282205
OI 0 Missing Rate: 0.8133952020480678
                             OI_0
patientunitstayid                
141203                   0.510000
141227                   1.777778
141296                   2.571429
141297                   1.180000
141313                   0.920000
...                           ...
3353147                  2.050000
3353201                  3.290000
3353213            1000000.000000
3353216                  2.260000
3353226                  3.528571

[34134 rows x 1 columns]


In [139]:
#temperature
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Temperature (C)') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
TP_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','nursingchartvalue'])
TP_size = TP_res.groupby("patientunitstayid").size()
print("Temperature max columns:" + str(max(TP_size)) + "\n" + "Temperature avg columns:" + str(sum(TP_size) / len(TP_size)) + "\n" + "Temperature missing rate:" + str((19615-len(TP_size))/19615))

#store the measure offset
TP_time = TP_res
#TP_time turn continuous data from vertical to horizontal
TP_time['cumCount'] = TP_time.groupby(['patientunitstayid']).cumcount()
TP_time = TP_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')
TP_time = TP_time.iloc[:,:length_TP]

#TP turn continuous data from vertical to horizontal
TP_res['cumCount'] = TP_res.groupby(['patientunitstayid']).cumcount()
TP = TP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartvalue')
TP = TP.iloc[:,:length_TP]

#find extreme and slope

#TP let format right
column = []
i = 0
while i < length_TP:
    column.append("TP_" + "%s"%str(i))
    i += 1
TP.columns = column
TP.fillna(1000000)
TP = pd.DataFrame(TP,dtype = np.float)

#find extreme
TP_time.columns = column
TP_sum = pd.DataFrame()
TP_sum['max_TP'] = TP.max(axis=1)
TP_sum['max_INDEX'] = TP.idxmax(axis=1)
TP_sum['min_TP'] = TP.min(axis=1,skipna = True)
TP_sum['min_INDEX'] = TP.idxmin(axis=1, skipna=True)
TP_sum['TP_delta'] = TP_sum['max_TP'] - TP_sum['min_TP']
TP_sum['TP_fever'] = 1*(TP_sum['max_TP'] >= 38)
ind = 0
TP_sum['slope'] = 0
for i in TP_sum.index:
    during = TP_time.loc[i,TP_sum['max_INDEX'].loc[i]] - TP_time.loc[i,TP_sum['min_INDEX'].loc[i]]
    during /= 60
    if during != 0:
        TP_sum['slope'].iloc[ind] = TP_sum['TP_delta'].iloc[ind] / during
    else:
        TP_sum['slope'].iloc[ind] = 1000000
    ind += 1
TP_sum.drop('max_INDEX',axis=1,inplace = True)
TP_sum.drop('min_INDEX',axis=1,inplace = True)
TP = TP.fillna(1000000)
TP_sum = TP_sum.fillna(1000000)

#caculate delta
TP_time.rename(columns={'TP_0':'TP_offset_0', 'TP_1':'TP_offset_1','TP_2':'TP_offset_2', 'TP_3':'TP_offset_3','TP_4':'TP_offset_4', 'TP_5':'TP_offset_5'}, inplace=True)
TP_time.fillna(1000000)
TP_delta = TP
TP_delta = pd.merge(TP_delta,TP_time,on='patientunitstayid',how='left')
i = 1
while i < length_TP:
    TP_delta['TP_delta_%s'%str(i)] = (TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_%s'%str(i)] - TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_%s'%str(i-1)]) \
    / ((TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_offset_%s'%str(i)] - TP_delta[TP_delta['TP_%s'%str(i)] < 1000000]['TP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_TP:
    TP_delta.drop(columns = ['TP_%s'%str(i)], inplace=True)
    TP_delta.drop(columns = ['TP_offset_%s'%str(i)], inplace=True)
    i += 1
TP_delta = TP_delta.fillna(1000000)
TP_delta = pd.DataFrame(TP_delta,dtype = np.float)

print("TP delta Missing Rate:",1 - TP_sum[TP_sum['TP_delta']!=1000000]['TP_delta'].count()/181244)
print("TP delta 1 Missing Rate:",1 - TP_delta[TP_delta['TP_delta_1']!=1000000]['TP_delta_1'].count()/181244)
print(TP,TP_delta,TP_sum)


Temperature max columns:138
Temperature avg columns:2.750843088297286
Temperature missing rate:-5.288860565893449
TP delta Missing Rate: 0.3193926419633202
TP delta 1 Missing Rate: 0.6064642139877734
                   TP_0       TP_1
patientunitstayid                 
141203             36.4       36.7
141227             37.6       37.2
141229             36.4  1000000.0
141284             36.6  1000000.0
141288             37.5       36.2
...                 ...        ...
3353216            34.9  1000000.0
3353226            36.7  1000000.0
3353235            36.6  1000000.0
3353237            37.4       36.8
3353263            37.1  1000000.0

[123356 rows x 2 columns]                        TP_delta_1
patientunitstayid                
141203                   0.068441
141227                  -0.050314
141229             1000000.000000
141284             1000000.000000
141288                  -3.545455
...                           ...
3353216            1000000.000000
3353226     

In [140]:
#urine
i = 0
offset = 360
NextOffset = 0
urine_dflist = []
for i in range(1,1 + int(int(strsql_timeoffst)/360)):
    PreOffset = NextOffset
    NextOffset = offset*i
    sql = "SELECT IO.patientunitstayid,IO.intakeoutputoffset,IO.cellvaluenumeric FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,intakeoutput IO WHERE TB.patientunitstayid=IO.patientunitstayid AND IO.celllabel IN('Urine') AND IO.intakeoutputoffset BETWEEN TB.diagnosisoffset-"+str(NextOffset)+" AND TB.diagnosisoffset-"+str(PreOffset)+" ORDER BY IO.patientunitstayid,IO.intakeoutputoffset"
    cur.execute(sql)
    urine_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','intakeoutputoffset_'+str(i),'urine_'+str(i)])
    urine_tdflist.append(urine_res.groupby(['patientunitstayid']).agg({'intakeoutputoffset_'+str(i):np.sum}).reset_index())
    urine_res = urine_res.groupby(['patientunitstayid']).agg({'urine_'+str(i):np.sum}).reset_index() 
    urine_dflist.append(urine_res)


urine = pd.DataFrame()
urine = urine_dflist[0]
for data in urine_dflist[1:]:
    urine = pd.merge(urine,data,on = 'patientunitstayid',how = 'left')
urine = urine.set_index('patientunitstayid')
urine = urine.fillna(1000000)
# print(len(urine.index))
i = 0
total = []
while i < len(urine.index):
    a = 0
    if urine.iloc[i]['urine_1'] != 1000000:
        a += float(urine.iloc[i]['urine_1'])
    if urine.iloc[i]['urine_2'] != 1000000:
        a += float(urine.iloc[i]['urine_2'])
    if urine.iloc[i]['urine_3'] != 1000000:
        a += float(urine.iloc[i]['urine_3'])
    if urine.iloc[i]['urine_4'] != 1000000:
        a += float(urine.iloc[i]['urine_4'])
    total.append(a)
    i += 1
urine['urine_total'] = total
urine = pd.DataFrame(urine,dtype = np.float)

# #caculate delta
# urine_time = pd.DataFrame()
# urine_time = urine_tdflist[0]
# for data in urine_tdflist[1:]:
#     urine_time = pd.merge(urine_time,data,on = 'patientunitstayid',how = 'left')
# urine_time = urine_time.set_index('patientunitstayid')
# urine_time = urine_time.fillna(1000000)
# urine_time = pd.DataFrame(urine_time,dtype = np.float)

# i = 2
# urine_delta = urine
# urine_delta = pd.merge(urine_delta,urine_time,on='patientunitstayid',how='left')
# while i < 5:
#     urine_delta['urine_delta_%s'%str(i-1)] = (urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['urine_%s'%str(i)] - urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['urine_%s'%str(i-1)]) \
#     / ((urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['intakeoutputoffset_%s'%str(i)] - urine_delta[urine_delta['urine_%s'%str(i)] < 1000000]['intakeoutputoffset_%s'%str(i-1)])/60)
#     i += 1
# i = 1
# while i < 5:
#     urine_delta.drop(columns = ['urine_%s'%str(i)], inplace=True)
#     urine_delta.drop(columns = ['intakeoutputoffset_%s'%str(i)], inplace=True)
#     i += 1
# urine_delta = urine_delta.fillna(1000000)
# urine_delta = pd.DataFrame(urine_delta,dtype = np.float)
urine.drop(columns = ['urine_1','urine_2','urine_3','urine_4'], inplace=True)
# urine_delta.drop(columns = ['urine_total'], inplace=True)
aki_urine = urine
# aki_urine_delta = urine_delta

print("urine total Missing Rate:",1 - urine[urine['urine_total']!=1000000]['urine_total'].count()/181244)
print(urine)



urine total Missing Rate: 0.7739566551168591
                   urine_total
patientunitstayid             
141392                   100.0
141631                   400.0
141800                  1550.0
141822                   500.0
141939                    50.0
...                        ...
3353198                   50.0
3353201                  725.0
3353213                 2000.0
3353216                 2500.0
3353237                 4500.0

[40969 rows x 1 columns]


In [141]:
#diastolic blood pressure(noninvasive)
cur.execute("SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivediastolic FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,observationoffset")
diastolicBP_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','observationoffset','noninvasivediastolic'])
diastolicBP_res = diastolicBP_res.groupby(['patientunitstayid','observationoffset']).agg({'noninvasivediastolic':np.min}).reset_index()
diastolicBP_size = diastolicBP_res.groupby("patientunitstayid").size()
print("diastolicBP max columns:" + str(max(diastolicBP_size)) + "\n" + "diastolicBP avg columns:" + str(sum(diastolicBP_size)/len(diastolicBP_size)) + "\n" + "diastolicBP missing rate:" + str((181244-len(diastolicBP_size))/181244))

#diastolicBP turn continuous data from vertical to horizontal
diastolicBP_res['cumCount'] = diastolicBP_res.groupby(['patientunitstayid']).cumcount()
diastolicBP = diastolicBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'noninvasivediastolic')
diastolicBP = diastolicBP.iloc[:,:length_diastolicBP]

#diastolicBP let format right
column = []
i = 0
while i < length_diastolicBP:
    column.append("diastolicBP_"+"%s"%str(i))
    i += 1
diastolicBP.columns = column
diastolicBP = diastolicBP.fillna(1000000)
diastolicBP = pd.DataFrame(diastolicBP,dtype = np.float)

#caculate delta

#store the measure offset
diastolicBP_time = diastolicBP_res
#diastolicBP_time turn continuous data from vertical to horizontal
diastolicBP_time['cumCount'] = diastolicBP_time.groupby(['patientunitstayid']).cumcount()
diastolicBP_time = diastolicBP_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'observationoffset')
diastolicBP_time = diastolicBP_time.iloc[:,:length_diastolicBP]
diastolicBP_time = diastolicBP_time.fillna(1000000)
column = []
i = 0
while i < length_diastolicBP:
    column.append("diastolicBP_offset_" + "%s"%str(i))
    i += 1
diastolicBP_time.columns = column

i = 1
diastolicBP_delta = diastolicBP
diastolicBP_delta = pd.merge(diastolicBP_delta,diastolicBP_time,on='patientunitstayid',how='left')
while i < length_diastolicBP:
    diastolicBP_delta['diastolicBP_delta_%s'%str(i)] = (diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_%s'%str(i)] - diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_%s'%str(i-1)]) \
    / ((diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_offset_%s'%str(i)] - diastolicBP_delta[diastolicBP_delta['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_diastolicBP:
    diastolicBP_delta.drop(columns = ['diastolicBP_%s'%str(i)], inplace=True)
    diastolicBP_delta.drop(columns = ['diastolicBP_offset_%s'%str(i)], inplace=True)
    i += 1
diastolicBP_delta = diastolicBP_delta.fillna(1000000)
diastolicBP_delta = pd.DataFrame(diastolicBP_delta,dtype = np.float)

print("diastolicBP delta 1 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_1']!=1000000]['diastolicBP_delta_1'].count()/181244)
print("diastolicBP delta 2 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_2']!=1000000]['diastolicBP_delta_2'].count()/181244)
print("diastolicBP delta 3 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_3']!=1000000]['diastolicBP_delta_3'].count()/181244)
print("diastolicBP delta 4 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_4']!=1000000]['diastolicBP_delta_4'].count()/181244)
print("diastolicBP delta 5 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_5']!=1000000]['diastolicBP_delta_5'].count()/181244)
print("diastolicBP delta 6 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_6']!=1000000]['diastolicBP_delta_6'].count()/181244)
print("diastolicBP delta 7 Missing Rate:",1 - diastolicBP_delta[diastolicBP_delta['diastolicBP_delta_7']!=1000000]['diastolicBP_delta_7'].count()/181244)
print(diastolicBP,diastolicBP_delta)


diastolicBP max columns:536
diastolicBP avg columns:5.655467323601611
diastolicBP missing rate:0.3684149544260776
diastolicBP delta 1 Missing Rate: 0.5139039085431794
diastolicBP delta 2 Missing Rate: 0.6325285250822097
diastolicBP delta 3 Missing Rate: 0.715196089249851
diastolicBP delta 4 Missing Rate: 0.7728531703118449
diastolicBP delta 5 Missing Rate: 0.8161815011807287
diastolicBP delta 6 Missing Rate: 0.8483867052150692
diastolicBP delta 7 Missing Rate: 0.8728840678863852
                   diastolicBP_0  diastolicBP_1  diastolicBP_2  diastolicBP_3  \
patientunitstayid                                                               
141203                      59.0           67.0           55.0           54.0   
141227                      67.0           66.0           55.0      1000000.0   
141229                      69.0      1000000.0      1000000.0      1000000.0   
141284                      58.0           62.0      1000000.0      1000000.0   
141288                      54

In [142]:
#systolic blood pressure(noninvasive)
cur.execute("SELECT VP.patientunitstayid,VP.observationoffset,VP.noninvasivesystolic FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,vitalaperiodic VP WHERE TB.patientunitstayid=VP.patientunitstayid AND noninvasivesystolic IS NOT NULL AND VP.observationoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,observationoffset")
systolicBP_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','observationoffset','noninvasivesystolic'])
systolicBP_res = systolicBP_res.groupby(['patientunitstayid','observationoffset']).agg({'noninvasivesystolic':np.min}).reset_index()
systolicBP_size = systolicBP_res.groupby("patientunitstayid").size()
print("systolicBP max columns:" + str(max(systolicBP_size)) + "\n" + "systolicBP avg columns:" + str(sum(systolicBP_size)/len(systolicBP_size)) + "\n" + "systolicBP missing rate:" + str((181244-len(systolicBP_size))/181244))

#systolicBP turn continuous data from vertical to horizontal
systolicBP_res['cumCount'] = systolicBP_res.groupby(['patientunitstayid']).cumcount()
systolicBP = systolicBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'noninvasivesystolic')
systolicBP = systolicBP.iloc[:,:length_systolicBP]

#systolicBP let format right
column = []
i = 0
while i < length_systolicBP:
    column.append("systolicBP_"+"%s"%str(i))
    i += 1
systolicBP.columns = column
systolicBP = systolicBP.fillna(1000000)
systolicBP = pd.DataFrame(systolicBP,dtype = np.float)

#caculate delta

#store the measure offset
systolicBP_time = systolicBP_res
#systolicBP_time turn continuous data from vertical to horizontal
systolicBP_time['cumCount'] = systolicBP_time.groupby(['patientunitstayid']).cumcount()
systolicBP_time = systolicBP_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'observationoffset')
systolicBP_time = systolicBP_time.iloc[:,:length_systolicBP]
systolicBP_time = systolicBP_time.fillna(1000000)
column = []
i = 0
while i < length_systolicBP:
    column.append("systolicBP_offset_" + "%s"%str(i))
    i += 1
systolicBP_time.columns = column

i = 1
systolicBP_delta = systolicBP 
systolicBP_delta = pd.merge(systolicBP_delta,systolicBP_time,on='patientunitstayid',how='left')
while i < length_systolicBP:
    systolicBP_delta['systolicBP_delta_%s'%str(i)] = (systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_%s'%str(i)] - systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_%s'%str(i-1)]) \
    / ((systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i)] - systolicBP_delta[systolicBP_delta['systolicBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_systolicBP:
    systolicBP_delta.drop(columns = ['systolicBP_%s'%str(i)], inplace=True)
    systolicBP_delta.drop(columns = ['systolicBP_offset_%s'%str(i)], inplace=True)
    i += 1
systolicBP_delta = systolicBP_delta.fillna(1000000)
systolicBP_delta = pd.DataFrame(systolicBP_delta,dtype = np.float)

print("systolicBP delta 1 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_1']!=1000000]['systolicBP_delta_1'].count()/181244)
print("systolicBP delta 2 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_2']!=1000000]['systolicBP_delta_2'].count()/181244)
print("systolicBP delta 3 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_3']!=1000000]['systolicBP_delta_3'].count()/181244)
print("systolicBP delta 4 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_4']!=1000000]['systolicBP_delta_4'].count()/181244)
print("systolicBP delta 5 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_5']!=1000000]['systolicBP_delta_5'].count()/181244)
print("systolicBP delta 6 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_6']!=1000000]['systolicBP_delta_6'].count()/181244)
print("systolicBP delta 7 Missing Rate:",1 - systolicBP_delta[systolicBP_delta['systolicBP_delta_7']!=1000000]['systolicBP_delta_7'].count()/181244)
print(systolicBP,systolicBP_delta)


systolicBP max columns:536
systolicBP avg columns:5.655467323601611
systolicBP missing rate:0.3684149544260776
systolicBP delta 1 Missing Rate: 0.5134735494692237
systolicBP delta 2 Missing Rate: 0.6321643750965549
systolicBP delta 3 Missing Rate: 0.7149478051687228
systolicBP delta 4 Missing Rate: 0.7726710953190175
systolicBP delta 5 Missing Rate: 0.8160214958840017
systolicBP delta 6 Missing Rate: 0.8482984264306681
systolicBP delta 7 Missing Rate: 0.8728123413740593
                   systolicBP_0  systolicBP_1  systolicBP_2  systolicBP_3  \
patientunitstayid                                                           
141203                     98.0         114.0          86.0          76.0   
141227                    159.0         131.0         100.0     1000000.0   
141229                    151.0     1000000.0     1000000.0     1000000.0   
141284                    130.0         126.0     1000000.0     1000000.0   
141288                     96.0     1000000.0     1000000.0    

In [143]:
#pulse pressure
pulseP_res = pd.DataFrame()
pulseP_res = pd.merge(diastolicBP_res,systolicBP_res,on = ['patientunitstayid','observationoffset'],how = 'inner')
pulseP_res['pulse pressure'] = pulseP_res['noninvasivesystolic'] - pulseP_res['noninvasivediastolic']
pulseP_size = pulseP_res.groupby("patientunitstayid").size()
print("pulse pressure max columns:" + str(max(pulseP_size)) + "\n" + "pulse pressure avg columns:" + str(sum(pulseP_size)/len(pulseP_size)) + "\n" + "pulse pressure missing rate:" + str((19615-len(pulseP_size))/19615))

#pulseP turn continuous data from vertical to horizontal
pulseP_res['cumCount'] = pulseP_res.groupby(['patientunitstayid']).cumcount()
pulseP = pulseP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'pulse pressure')
pulseP = pulseP.iloc[:,:length_pulseP]

#pulseP let format right
column = []
i = 0
while i < length_pulseP:
    column.append("pulseP_"+"%s"%str(i))
    i += 1
pulseP.columns = column
pulseP = pulseP.fillna(1000000)
pulseP = pd.DataFrame(pulseP,dtype = np.float)

print("pulseP 1 Missing Rate:",1 - pulseP[pulseP['pulseP_1']!=1000000]['pulseP_1'].count()/181244)
print("pulseP 2 Missing Rate:",1 - pulseP[pulseP['pulseP_2']!=1000000]['pulseP_2'].count()/181244)
print("pulseP 3 Missing Rate:",1 - pulseP[pulseP['pulseP_3']!=1000000]['pulseP_3'].count()/181244)
print("pulseP 4 Missing Rate:",1 - pulseP[pulseP['pulseP_4']!=1000000]['pulseP_4'].count()/181244)
print("pulseP 5 Missing Rate:",1 - pulseP[pulseP['pulseP_5']!=1000000]['pulseP_5'].count()/181244)
print("pulseP 6 Missing Rate:",1 - pulseP[pulseP['pulseP_6']!=1000000]['pulseP_6'].count()/181244)
print("pulseP 7 Missing Rate:",1 - pulseP[pulseP['pulseP_7']!=1000000]['pulseP_7'].count()/181244)
print(pulseP)


pulse pressure max columns:536
pulse pressure avg columns:5.655467323601611
pulse pressure missing rate:-4.835890899821565
pulseP 1 Missing Rate: 0.5139039085431794
pulseP 2 Missing Rate: 0.6325285250822097
pulseP 3 Missing Rate: 0.715196089249851
pulseP 4 Missing Rate: 0.7728531703118449
pulseP 5 Missing Rate: 0.8161815011807287
pulseP 6 Missing Rate: 0.8483867052150692
pulseP 7 Missing Rate: 0.8728840678863852
                   pulseP_0   pulseP_1   pulseP_2   pulseP_3   pulseP_4  \
patientunitstayid                                                         
141203                 39.0       47.0       31.0       22.0       30.0   
141227                 92.0       65.0       45.0  1000000.0  1000000.0   
141229                 82.0  1000000.0  1000000.0  1000000.0  1000000.0   
141284                 72.0       64.0  1000000.0  1000000.0  1000000.0   
141288                 42.0  1000000.0  1000000.0  1000000.0  1000000.0   
...                     ...        ...        ...        ..

In [144]:
#mean blood pressure(noninvasive)
meanBP = pulseP
meanBP = pd.merge(meanBP,diastolicBP,on='patientunitstayid',how='left')
i = 0
while i < length_diastolicBP:
    meanBP['meanBP_%s'%str(i)] = ((meanBP[meanBP['pulseP_%s'%str(i)] < 1000000]['pulseP_%s'%str(i)]/3) + meanBP[meanBP['diastolicBP_%s'%str(i)] < 1000000]['diastolicBP_%s'%str(i)])
    i += 1
i = 0
while i < length_systolicBP:
    meanBP.drop(columns = ['pulseP_%s'%str(i)], inplace=True)
    meanBP.drop(columns = ['diastolicBP_%s'%str(i)], inplace=True)
    i += 1

meanBP = meanBP.fillna(1000000)
#caculate delta

#store the measure offset
i = 1
meanBP_delta = meanBP
meanBP_delta = pd.merge(meanBP_delta,systolicBP_time,on='patientunitstayid',how='left')
while i < length_systolicBP:
    meanBP_delta['meanBP_delta_%s'%str(i)] = (meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['meanBP_%s'%str(i)] - meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['meanBP_%s'%str(i-1)]) \
    / ((meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i)] - meanBP_delta[meanBP_delta['meanBP_%s'%str(i)] < 1000000]['systolicBP_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_systolicBP:
    meanBP_delta.drop(columns = ['meanBP_%s'%str(i)], inplace=True)
    meanBP_delta.drop(columns = ['systolicBP_offset_%s'%str(i)], inplace=True)
    i += 1
meanBP_delta = meanBP_delta.fillna(1000000)
meanBP_delta = pd.DataFrame(meanBP_delta,dtype = np.float)

print("meanBP 0 Missing Rate:",1 - meanBP[meanBP['meanBP_0']!=1000000]['meanBP_0'].count()/19615)
print("meanBP delta 1 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_1']!=1000000]['meanBP_delta_1'].count()/181244)
print("meanBP delta 2 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_2']!=1000000]['meanBP_delta_2'].count()/181244)
print("meanBP delta 3 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_3']!=1000000]['meanBP_delta_3'].count()/181244)
print("meanBP delta 4 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_4']!=1000000]['meanBP_delta_4'].count()/181244)
print("meanBP delta 5 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_5']!=1000000]['meanBP_delta_5'].count()/181244)
print("meanBP delta 6 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_6']!=1000000]['meanBP_delta_6'].count()/181244)
print("meanBP delta 7 Missing Rate:",1 - meanBP_delta[meanBP_delta['meanBP_delta_7']!=1000000]['meanBP_delta_7'].count()/181244)
print(meanBP,meanBP_delta)


meanBP 0 Missing Rate: -4.830486872291614
meanBP delta 1 Missing Rate: 0.5139039085431794
meanBP delta 2 Missing Rate: 0.6325285250822097
meanBP delta 3 Missing Rate: 0.715196089249851
meanBP delta 4 Missing Rate: 0.7728531703118449
meanBP delta 5 Missing Rate: 0.8161815011807287
meanBP delta 6 Missing Rate: 0.8483867052150692
meanBP delta 7 Missing Rate: 0.8728840678863852
                     meanBP_0        meanBP_1        meanBP_2        meanBP_3  \
patientunitstayid                                                               
141203              72.000000       82.666667       65.333333       61.333333   
141227              97.666667       87.666667       70.000000  1000000.000000   
141229              96.333333  1000000.000000  1000000.000000  1000000.000000   
141284              82.000000       83.333333  1000000.000000  1000000.000000   
141288              68.000000  1000000.000000  1000000.000000  1000000.000000   
...                       ...             ...           

In [145]:
#age,since its missing rate is 0,so I don't print its relative information
cur.execute("SELECT patientunitstayid,age FROM patient WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
age_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','age'])
age = age_res
age = np.array(age)
age = age.tolist()
for i in age:
    if i[1].isdigit() == False:
        i[1] = ''.join([x for x in i[1] if x.isdigit()])
        if i[1] != '':
            i[1] = int(i[1])
        else:
            i[1] = 1000000
    else:
        i[1] = int(i[1])
age = pd.DataFrame(age)
age.columns = ['patientunitstayid','age']
age.set_index('patientunitstayid',inplace=True)
age = pd.DataFrame(age,dtype = np.float)

print(age)


                    age
patientunitstayid      
141168             70.0
141178             52.0
141179             52.0
141194             68.0
141196             71.0
...                 ...
3353216            50.0
3353226            79.0
3353235            50.0
3353237            79.0
3353263            35.0

[181244 rows x 1 columns]


In [146]:
#weight
cur.execute("SELECT patientunitstayid,admissionweight FROM patient WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) AND admissionweight IS NOT NULL ORDER BY patientunitstayid")
weight_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','weight'])
weight_size = weight_res.groupby("patientunitstayid").size()
print("weight max columns:" + str(max(weight_size)) + "\n" + "weight avg columns:" + str(sum(weight_size)/len(weight_size)) + "\n" + "weight missing rate:" + str((181244-len(weight_size))/181244))

#weight turn continuous data from vertical to horizontal
weight_res['cumCount'] = weight_res.groupby(['patientunitstayid']).cumcount()
weight = weight_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'weight')
# weight = weight.iloc[:,:length_weight]

#weight let format right
weight = weight.fillna(1000000)
weight = pd.DataFrame(weight,dtype = np.float)
weight.columns = ['weight']
aki_weight = weight

print(weight)


weight max columns:1
weight avg columns:1.0
weight missing rate:0.08788152987133367
                   weight
patientunitstayid        
141168               84.3
141178               54.4
141194               73.9
141197              102.1
141203               70.2
...                   ...
3353216              55.4
3353226              58.4
3353235              90.0
3353237              78.4
3353263             100.8

[165316 rows x 1 columns]


In [147]:
#albumin
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('albumin') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
albumin_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
albumin_res = albumin_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
albumin_size = albumin_res.groupby("patientunitstayid").size()
print("albumin max columns:" + str(max(albumin_size)) + "\n" + "albumin avg columns:" + str(sum(albumin_size)/len(albumin_size)) + "\n" + "albumin missing rate:" + str((181244-len(albumin_size))/181244))

#albumin turn continuous data from vertical to horizontal
albumin_res['cumCount'] = albumin_res.groupby(['patientunitstayid']).cumcount()
albumin = albumin_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
albumin = albumin.iloc[:,:length_albumin]

#albumin let format right
column = []
i = 0
while i < length_albumin:
    column.append("albumin_"+"%s"%str(i))
    i += 1
albumin.columns = column
albumin = albumin.fillna(1000000)
albumin = pd.DataFrame(albumin,dtype = np.float)

#caculate delta

#store the measure offset
albumin_time = albumin_res
#albumin_time turn continuous data from vertical to horizontal
albumin_time['cumCount'] = albumin_time.groupby(['patientunitstayid']).cumcount()
albumin_time = albumin_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
albumin_time = albumin_time.iloc[:,:length_albumin]
albumin_time = albumin_time.fillna(1000000)
column = []
i = 0
while i < length_albumin:
    column.append("albumin_offset_" + "%s"%str(i))
    i += 1
albumin_time.columns = column

i = 1
albumin_delta = albumin
albumin_delta = pd.merge(albumin_delta,albumin_time,on='patientunitstayid',how='left')
while i < length_albumin:
    albumin_delta['albumin_delta_%s'%str(i)] = (albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_%s'%str(i)] - albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_%s'%str(i-1)]) \
    / ((albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_offset_%s'%str(i)] - albumin_delta[albumin_delta['albumin_%s'%str(i)] < 1000000]['albumin_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_albumin:
    albumin_delta.drop(columns = ['albumin_%s'%str(i)], inplace=True)
    albumin_delta.drop(columns = ['albumin_offset_%s'%str(i)], inplace=True)
    i += 1
albumin_delta = albumin_delta.fillna(1000000)
albumin_delta = pd.DataFrame(albumin_delta,dtype = np.float)

print("albumin delta 1 Missing Rate:",1 - albumin_delta[albumin_delta['albumin_delta_1']!=1000000]['albumin_delta_1'].count()/181244)
print(albumin,albumin_delta)


albumin max columns:5
albumin avg columns:1.0847455195075404
albumin missing rate:0.6154852022687648
albumin delta 1 Missing Rate: 0.9693948489329302
                   albumin_0  albumin_1
patientunitstayid                      
141203                   1.4  1000000.0
141229                   3.1  1000000.0
141288                   3.0  1000000.0
141296                   2.1  1000000.0
141313                   2.7  1000000.0
...                      ...        ...
3353140                  3.7  1000000.0
3353145                  2.6  1000000.0
3353147                  4.6  1000000.0
3353213                  3.4  1000000.0
3353216                  2.8  1000000.0

[69691 rows x 2 columns]                    albumin_delta_1
patientunitstayid                 
141203                   1000000.0
141229                   1000000.0
141288                   1000000.0
141296                   1000000.0
141313                   1000000.0
...                            ...
3353140                 

In [148]:
#creatinine
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('creatinine') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
creatinine_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','labresultoffset','labresult'])
creatinine_res=creatinine_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
creatinine_size=creatinine_res.groupby("patientunitstayid").size()
print("creatinine max columns:" + str(max(creatinine_size)) + "\n" + "creatinine avg columns:" + str(sum(creatinine_size)/len(creatinine_size)) + "\n" + "creatinine missing rate:" + str((181244-len(creatinine_size))/181244))

#creatinine turn continuous data from vertical to horizontal
creatinine_res['cumCount'] = creatinine_res.groupby(['patientunitstayid']).cumcount()
creatinine = creatinine_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
creatinine = creatinine.iloc[:,:length_creatinine]

#creatinine let format right
column = []
i = 0
while i < length_creatinine:
    column.append("creatinine_"+"%s"%str(i))
    i += 1
creatinine.columns = column
creatinine = creatinine.fillna(1000000)
creatinine = pd.DataFrame(creatinine,dtype = np.float)

#caculate delta

#store the measure offset
creatinine_time = creatinine_res
#creatinine_time turn continuous data from vertical to horizontal
creatinine_time['cumCount'] = creatinine_time.groupby(['patientunitstayid']).cumcount()
creatinine_time = creatinine_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
creatinine_time = creatinine_time.iloc[:,:length_creatinine]
creatinine_time = creatinine_time.fillna(1000000)
column = []
i = 0
while i < length_creatinine:
    column.append("creatinine_offset_" + "%s"%str(i))
    i += 1
creatinine_time.columns = column

i = 1
creatinine_delta = creatinine
creatinine_delta = pd.merge(creatinine_delta,creatinine_time,on='patientunitstayid',how='left')
while i < length_creatinine:
    creatinine_delta['creatinine_delta_%s'%str(i)] = (creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_%s'%str(i)] - creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_%s'%str(i-1)]) \
    / ((creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_offset_%s'%str(i)] - creatinine_delta[creatinine_delta['creatinine_%s'%str(i)] < 1000000]['creatinine_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_creatinine:
    creatinine_delta.drop(columns = ['creatinine_%s'%str(i)], inplace=True)
    creatinine_delta.drop(columns = ['creatinine_offset_%s'%str(i)], inplace=True)
    i += 1
creatinine_delta = creatinine_delta.fillna(1000000)
creatinine_delta = pd.DataFrame(creatinine_delta,dtype = np.float)

print("creatinine delta 1 Missing Rate:",1 - creatinine_delta[creatinine_delta['creatinine_delta_1']!=1000000]['creatinine_delta_1'].count()/181244)
print(creatinine,creatinine_delta)


creatinine max columns:11
creatinine avg columns:1.2418939879443098
creatinine missing rate:0.40595550749266185
creatinine delta 1 Missing Rate: 0.8791684138509412
                   creatinine_0  creatinine_1
patientunitstayid                            
141203                     0.39          0.56
141227                     1.40          1.50
141229                     1.36    1000000.00
141288                     0.95    1000000.00
141296                     5.67    1000000.00
...                         ...           ...
3353213                    0.50    1000000.00
3353216                    0.73    1000000.00
3353226                    4.88    1000000.00
3353235                    1.04    1000000.00
3353263                    1.06    1000000.00

[107667 rows x 2 columns]                    creatinine_delta_1
patientunitstayid                    
141203                       0.021519
141227                       0.052174
141229                 1000000.000000
141288               

In [149]:
#gfr

#height
cur.execute("SELECT patientunitstayid,admissionheight FROM patient WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) AND admissionheight IS NOT NULL ORDER BY patientunitstayid")
height_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','height'])
height_size=height_res.groupby("patientunitstayid").size()
print("height max columns:" + str(max(height_size)) + "\n" + "height avg columns:" + str(sum(height_size)/len(height_size)) + "\n" + "height missing rate:" + str((181244-len(height_size))/181244))

#GFR-related
cur.execute("SELECT patientunitstayid,age,gender,ethnicity FROM patient WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
GFR_res = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','age','gender','ethnicity'])

GFR_res = pd.merge(GFR_res,height_res,on = 'patientunitstayid',how = 'left')
GFR_res = GFR_res.fillna(1000000)
#turn dataframe into list
GFR_res = np.array(GFR_res)
GFR_res = GFR_res.tolist()

for i in GFR_res:
    if i[3] == "African American":
        i[3] = 1
    else:
        i[3] = 0
for i in GFR_res:
    if i[4] > 60:
        i[4] = float(i[4])
    else:
        if i[4] < 2:
            i[4] = 100*float(i[4])
        else:
            i[4] = -1
for i in GFR_res:
    if i[2] == "Female":
        i[2] = 1
    else:
        i[2] = 0
for i in GFR_res:
    if i[1].isdigit() == False:
        i[1] = ''.join([x for x in i[1] if x.isdigit()])
        if i[1] != '':
            i[1] = int(i[1])
        else:
            i[1] = -1
    else:
        i[1] = int(i[1])

GFR_res = pd.DataFrame(GFR_res)
GFR_res.columns = ['patientunitstayid','age','gender','ethnicity','height']
GFR_res.set_index('patientunitstayid', inplace=True)
GFR_res = pd.DataFrame(GFR_res,dtype = np.float)

i = 0
gfr = pd.merge(GFR_res,creatinine,on='patientunitstayid',how='left')
gfr = pd.merge(gfr,BUN,on='patientunitstayid',how='left')
gfr = pd.merge(gfr,albumin,on='patientunitstayid',how='left')
column = ['age','gender','ethnicity','height']
i = 0
while i < length_creatinine:
    column.append("creatinine_"+str("%s"%i))
    i += 1
i = 0
while i < length_BUN:
    column.append("BUN_"+str("%s"%i))
    i += 1
i = 0
while i < length_albumin:
    column.append("albumin_"+str("%s"%i))
    i += 1
gfr.columns = column

#clean the None value
i = 0
gfr.fillna(1000000)
while i < length_creatinine:
    clean_z = gfr["creatinine_"+"%s"%str(i)].fillna(1000000)
    clean_z[clean_z==""] = 1000000
    gfr["creatinine_"+"%s"%str(i)] = clean_z
    i += 1

#gfr MDRD
gfr['age2'] = gfr['age']
gfr['creatinine_02'] = gfr['creatinine_0']
gfr['gender2'] = gfr['gender']
gfr['ethnicity2'] = gfr['ethnicity']
gfr['height2'] = gfr['height']

#age under 17
gfr['gfr_0_C'] = 0.413 * gfr[(gfr['age2'] > 0) & (gfr['age2'] < 17) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['height2'] > 0) & (gfr['height2'] < 300)].height \
/ gfr[(gfr['age2'] > 0) & (gfr['age2'] < 17) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['height2'] > 0) & (gfr['height2'] < 300)].creatinine_0 / 10

#age beyond 17

#men not black
gfr['gfr_0_MNB'] = 186 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 0)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 0) ].age**(-0.203))

#women not black
gfr['gfr_0_WNB'] = 186 * 0.742 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 0)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 0) ].age**(-0.203))

#men black
gfr['gfr_0_MB'] = 186 * 1.210 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 1)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 0) & (gfr['ethnicity2'] == 1) ].age**(-0.203))

#women black
gfr['gfr_0_WB'] = 186 * 1.210 * 0.742 * (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 1)].creatinine_0**(-1.154)) \
* (gfr[(gfr['age2'] > 16) & (gfr['age2'] < 120) & (gfr['creatinine_02'] > 0) & (gfr['creatinine_02'] < 5) & (gfr['gender2'] == 1) & (gfr['ethnicity2'] == 1) ].age**(-0.203))

gfr = gfr.fillna(0)
#gfr.reset_index(inplace = True)
print()
gfr['gfr'] = gfr['gfr_0_WB'] + gfr['gfr_0_MB'] + gfr['gfr_0_WNB'] + gfr['gfr_0_MNB'] + gfr['gfr_0_C']
gfr[gfr['gfr'] == 0] = 1000000
gfr = pd.DataFrame(gfr,dtype = np.float)

gfr.drop(columns = ['age2','gender2','ethnicity2','height2','creatinine_02'], inplace=True)
gfr.drop(columns = ['gfr_0_WB','gfr_0_MB','gfr_0_WNB','gfr_0_MNB','gfr_0_C'], inplace=True)

for i in gfr[gfr['gfr'] > 180].index.values:
    gfr.loc[i]['gfr'] = 1000000

print("gfr Missing Rate :",1 - gfr[gfr['gfr'] != 1000000]['gfr'].count()/19615)
# print(gfr[(gfr['gfr'] != 1000000) & (gfr['gfr'] >200)]['gfr'])

gfr['age1'] = gfr['age']
gfr['creatinine_01'] = gfr['creatinine_0']
gfr['gender1'] = gfr['gender']
gfr['ethnicity1'] = gfr['ethnicity']

#gfr CKD_EPI

#men not black
gfr['gfr_0_MNB'] = 141 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.9) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.9)**(-0.411)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_MNB1'] = 141 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.9) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.9)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

#women not black
gfr['gfr_0_WNB'] = 144 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.7) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.7)**(-0.329)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_WNB1'] = 144 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.7) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0)].creatinine_0 / 0.7)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 0) ].age))

#men black
gfr['gfr_0_MB'] = 163 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.9) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.9)**(-0.411)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_MB1'] = 163 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.9) & (gfr['creatinine_01'] <= 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.9)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

#women black
gfr['gfr_0_WB'] = 166 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] <= 0.7) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.7)**(-0.329)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr['gfr_0_WB1'] = 166 * ((gfr[(gfr['age1'] > 0) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0.7) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 1) & (gfr['ethnicity1'] == 1)].creatinine_0 / 0.7)**(-1.209)) \
* (0.993**(gfr[(gfr['age1'] > 16) & (gfr['age1'] < 120) & (gfr['creatinine_01'] > 0) & (gfr['creatinine_01'] < 5) & (gfr['gender1'] == 0) & (gfr['ethnicity1'] == 0) ].age))

gfr = gfr.fillna(0)
#gfr.reset_index(inplace = True)
gfr['gfr_CKI'] = gfr['gfr_0_WB'] + gfr['gfr_0_MB'] + gfr['gfr_0_WNB'] + gfr['gfr_0_MNB'] + gfr['gfr_0_WB1'] + gfr['gfr_0_MB1'] + gfr['gfr_0_WNB1'] + gfr['gfr_0_MNB1']
gfr[gfr['gfr_CKI'] == 0] = 1000000
gfr = pd.DataFrame(gfr,dtype = np.float)

gfr.drop(columns = ['age1','gender1','ethnicity1','creatinine_01'], inplace=True)
gfr.drop(columns = ['gfr_0_WB','gfr_0_MB','gfr_0_WNB','gfr_0_MNB','gfr_0_WB1','gfr_0_MB1','gfr_0_WNB1','gfr_0_MNB1'], inplace=True)

for i in gfr[gfr['gfr_CKI'] > 180].index.values:
    gfr.loc[i]['gfr_CKI'] = 1000000

print("gfr_CKI Missing Rate :",1 - gfr[gfr['gfr_CKI'] != 1000000]['gfr_CKI'].count()/19615)


#gfr MDRD fine

#age under 17
gfr['gfr_0_C'] = 0.412 * gfr[(gfr['age'] > 0) & (gfr['age'] < 17) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['height'] > 0) & (gfr['height'] < 300)].height \
/ gfr[(gfr['age'] > 0) & (gfr['age'] < 17) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['height'] > 0) & (gfr['height'] < 300)].creatinine_0  / 10

#age beyond 17

#men not black
gfr['gfr_0_MNB'] = (170 * (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318)))

gfr['gfr_0_WNB'] = (170 * 0.762 *(gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 0) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318)))

#men black
gfr['gfr_0_MB'] = 170 * 1.18 * (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 0) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318))

#women black
gfr['gfr_0_WB'] = 170 * 1.18 * 0.762 *(gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].creatinine_0**(-0.999)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].age**(-0.176)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].BUN_0**(-0.170)) \
* (gfr[(gfr['age'] > 16) & (gfr['age'] < 120) & (gfr['creatinine_0'] > 0) & (gfr['creatinine_0'] < 5) & (gfr['gender'] == 1) & (gfr['ethnicity'] == 1) & (gfr['BUN_0'] > 0) & (gfr['BUN_0'] < 1000000) & (gfr['albumin_0'] > 0) & (gfr['albumin_0'] < 10)].albumin_0**(0.318))

gfr = gfr.fillna(0)
#gfr.reset_index(inplace = True)
gfr['gfr_fine'] = gfr['gfr_0_WB'] + gfr['gfr_0_MB'] + gfr['gfr_0_WNB'] + gfr['gfr_0_MNB'] + gfr['gfr_0_C']
gfr[gfr['gfr_fine'] == 0] = 1000000
gfr = pd.DataFrame(gfr,dtype = np.float)

gfr.drop(columns = ['age','gender','ethnicity','height','creatinine_0','creatinine_1','BUN_0','BUN_1','albumin_0','albumin_1'], inplace=True)
gfr.drop(columns = ['gfr_0_WB','gfr_0_MB','gfr_0_WNB','gfr_0_MNB','gfr_0_C'], inplace=True)
aki_gfr = gfr

for i in gfr[gfr['gfr_fine'] > 180].index.values:
    gfr.loc[i]['gfr_fine'] = 1000000

print("gfr_fine Missing Rate :",1 - gfr[gfr['gfr_fine'] != 1000000]['gfr_fine'].count()/19615)
print(gfr)


height max columns:1
height avg columns:1.0
height missing rate:0.022284875637262475

gfr Missing Rate : -4.149069589599796
gfr_CKI Missing Rate : -3.697425439714504
gfr_fine Missing Rate : -1.8633188886056589
                         gfr    gfr_CKI   gfr_fine
patientunitstayid                                 
141168             1000000.0  1000000.0  1000000.0
141178             1000000.0  1000000.0  1000000.0
141179             1000000.0  1000000.0  1000000.0
141194             1000000.0  1000000.0  1000000.0
141196             1000000.0  1000000.0  1000000.0
...                      ...        ...        ...
3353216            1000000.0  1000000.0  1000000.0
3353226            1000000.0  1000000.0  1000000.0
3353235            1000000.0  1000000.0  1000000.0
3353237            1000000.0  1000000.0  1000000.0
3353263            1000000.0  1000000.0  1000000.0

[181244 rows x 3 columns]


In [150]:
#condition-related
cur.execute("SELECT patientunitstayid,cirrhosis,gender,hepaticFailure,ventday1,diabetes FROM apachepredvar WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) ORDER BY patientunitstayid")
condition = pd.DataFrame(cur.fetchall(),columns=['patientunitstayid','cirrhosis','gender','hepaticFailure','ventday1','diabetes'])
condition = condition.fillna(1000000)
condition.set_index('patientunitstayid',inplace=True)

print(condition)
print("cirrhosis Missing Rate:",1 - condition[condition['cirrhosis']!=1000000]['cirrhosis'].count()/181244)
print("gender Missing Rate:",1 - condition[condition['gender']!=1000000]['gender'].count()/181244)
print("hepaticFailure Missing Rate:",1 - condition[condition['hepaticFailure']!=1000000]['hepaticFailure'].count()/181244)
print("ventday1 Missing Rate:",1 - condition[condition['ventday1']!=1000000]['ventday1'].count()/181244)
print("diabetes Missing Rate:",1 - condition[condition['diabetes']!=1000000]['diabetes'].count()/181244)


                   cirrhosis  gender  hepaticFailure  ventday1  diabetes
patientunitstayid                                                       
141168                     0       1               0         0         0
141178                     0       1               0         0         0
141194                     0       0               0         0         1
141197                     0       0               0         0         0
141203                     0       1               0         1         1
...                      ...     ...             ...       ...       ...
3353216                    0       1               0         1         0
3353226                    0       1               0         1         1
3353235                    0       0               0         0         0
3353237                    0       1               0         0         0
3353263                    0       0               0         0         0

[152614 rows x 5 columns]
cirrhosis Missing Rate: 

In [151]:
#glucose
cur.execute("SELECT L.patientunitstayid,L.labresultoffset,L.labresult FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,Lab L WHERE TB.patientunitstayid=L.patientunitstayid AND L.labname IN('glucose') AND L.labresultoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY L.patientunitstayid,L.labresultoffset")
glucose_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','labresultoffset','labresult'])
glucose_res = glucose_res.groupby(['patientunitstayid','labresultoffset']).agg({'labresult':np.min}).reset_index()
glucose_size = glucose_res.groupby("patientunitstayid").size()
print("glucose max columns:" + str(max(glucose_size)) + "\n" + "glucose avg columns:" + str(sum(glucose_size)/len(glucose_size)) + "\n" + "glucose missing rate:" + str((181244-len(glucose_size))/181244))

#glucose turn continuous data from vertical to horizontal
glucose_res['cumCount'] = glucose_res.groupby(['patientunitstayid']).cumcount()
glucose = glucose_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresult')
glucose = glucose.iloc[:,:length_glucose]

#glucose let format right
column = []
i = 0
while i < length_glucose:
    column.append("glucose_"+"%s"%str(i))
    i += 1
glucose.columns = column
glucose = glucose.fillna(1000000)
glucose = pd.DataFrame(glucose,dtype = np.float)

#caculate delta

#store the measure offset
glucose_time = glucose_res
#glucose_time turn continuous data from vertical to horizontal
glucose_time['cumCount'] = glucose_time.groupby(['patientunitstayid']).cumcount()
glucose_time = glucose_time.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'labresultoffset')
glucose_time = glucose_time.iloc[:,:length_glucose]
glucose_time = glucose_time.fillna(1000000)
column = []
i = 0
while i < length_glucose:
    column.append("glucose_offset_" + "%s"%str(i))
    i += 1
glucose_time.columns = column

i = 1
glucose_delta = glucose
glucose_delta = pd.merge(glucose_delta,glucose_time,on='patientunitstayid',how='left')
while i < length_glucose:
    glucose_delta['glucose_delta_%s'%str(i)] = (glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_%s'%str(i)] - glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_%s'%str(i-1)]) \
    / ((glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_offset_%s'%str(i)] - glucose_delta[glucose_delta['glucose_%s'%str(i)] < 1000000]['glucose_offset_%s'%str(i-1)])/60)
    i += 1
i = 0
while i < length_glucose:
    glucose_delta.drop(columns = ['glucose_%s'%str(i)], inplace=True)
    glucose_delta.drop(columns = ['glucose_offset_%s'%str(i)], inplace=True)
    i += 1
glucose_delta = glucose_delta.fillna(1000000)
glucose_delta = pd.DataFrame(glucose_delta,dtype = np.float)

print(glucose,glucose_delta)
print("glucose delta 1 Missing Rate:",1 - glucose_delta[glucose_delta['glucose_delta_1']!=1000000]['glucose_delta_1'].count()/181244)


glucose max columns:31
glucose avg columns:1.3994212499883691
glucose missing rate:0.40702588775352566
                   glucose_0  glucose_1
patientunitstayid                      
141203                  95.0      145.0
141227                 110.0      157.0
141229                 271.0  1000000.0
141288                  84.0  1000000.0
141296                  95.0  1000000.0
...                      ...        ...
3353213                100.0  1000000.0
3353216                167.0  1000000.0
3353226                175.0  1000000.0
3353235                100.0  1000000.0
3353263                100.0  1000000.0

[107473 rows x 2 columns]                    glucose_delta_1
patientunitstayid                 
141203                    6.329114
141227                   24.521739
141229              1000000.000000
141288              1000000.000000
141296              1000000.000000
...                            ...
3353213             1000000.000000
3353216             1000000.000000


In [152]:
cur.execute("SELECT T.patientunitstayid,T.treatmentoffset,T.treatmentstring FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,treatment T WHERE TB.patientunitstayid=T.patientunitstayid AND treatmentstring LIKE '%renal|dialysis%' AND T.treatmentoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY patientunitstayid,treatmentoffset")
dialysis_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','treatmentoffset','treatmentstring'])
dialysis_res = dialysis_res.groupby(['patientunitstayid']).agg({'treatmentoffset':np.min}).reset_index()
dialysis_res['treatmentoffset'] = 1
dialysis_res = dialysis_res.rename(columns = {'treatmentoffset':'IsDialysis'})
dialysis_res = pd.merge(akiDiagdf['patientunitstayid'],dialysis_res,on = 'patientunitstayid',how = 'left')
dialysis_res = dialysis_res.fillna(0)
dialysis = dialysis_res
dialysis.set_index('patientunitstayid',inplace=True)
print(dialysis)


                   IsDialysis
patientunitstayid            
141168                    0.0
141203                    0.0
141227                    0.0
141229                    0.0
141284                    0.0
...                       ...
3353216                   0.0
3353226                   0.0
3353235                   0.0
3353237                   0.0
3353263                   0.0

[153494 rows x 1 columns]


In [153]:
#MSI modified shock index
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Heart Rate') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
HR_res = pd.DataFrame(cur.fetchall(),columns = ['patientunitstayid','nursingchartoffset','HR_nursingchartvalue'])
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Non-Invasive BP Systolic') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
SBP_res = pd.DataFrame(cur.fetchall(),columns =[ 'patientunitstayid','nursingchartoffset','SBP_nursingchartvalue'])
cur.execute("SELECT N.patientunitstayid,N.nursingchartoffset,N.nursingchartvalue FROM (SELECT MIN(patientunitstayid) AS patientunitstayid,MIN(diagnosisoffset) AS diagnosisoffset FROM diagnosis WHERE patientunitstayid NOT IN(SELECT MAX(patientunitstayid) AS patientunitstayid FROM diagnosis WHERE diagnosisstring LIKE '%acute renal failure%' GROUP BY patientunitstayid ORDER BY patientunitstayid) GROUP BY patientunitstayid ORDER BY patientunitstayid) TB,nursecharting N WHERE TB.patientunitstayid=N.patientunitstayid AND N.nursingchartcelltypevalname IN('Non-Invasive BP Diastolic') AND N.nursingchartoffset BETWEEN TB.diagnosisoffset-"+strsql_timeoffst+" AND TB.diagnosisoffset ORDER BY N.patientunitstayid,N.nursingchartoffset")
DBP_res = pd.DataFrame(cur.fetchall(),columns =[ 'patientunitstayid','nursingchartoffset','DBP_nursingchartvalue'])

# time offset
HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
HR_time = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
SBP_time = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

DBP_res['cumCount'] = DBP_res.groupby(['patientunitstayid']).cumcount()
DBP_time = DBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'nursingchartoffset')

#real value
HR_res['cumCount'] = HR_res.groupby(['patientunitstayid']).cumcount()
HR = HR_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'HR_nursingchartvalue')

SBP_res['cumCount'] = SBP_res.groupby(['patientunitstayid']).cumcount()
SBP = SBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'SBP_nursingchartvalue')

DBP_res['cumCount'] = DBP_res.groupby(['patientunitstayid']).cumcount()
DBP = DBP_res.pivot(index = 'patientunitstayid',columns = 'cumCount',values = 'DBP_nursingchartvalue')

HR = HR.fillna(1000000)
SBP = SBP.fillna(1000000)
DBP = DBP.fillna(1000000)
count = 0

i = 0
column = []
while i < len(HR.columns):
    column.append("HR_%s"%i)
    i += 1
HR.columns = column
# print(HR)

i = 0
column = []
while i < len(SBP.columns):
    column.append("SBP_%s"%i)
    i += 1
SBP.columns = column
# print(SBP)

i = 0
column = []
while i < len(DBP.columns):
    column.append("DBP_%s"%i)
    i += 1
DBP.columns = column

HR_time = HR_time.fillna(1000000)
SBP_time = SBP_time.fillna(1000000)
DBP_time = DBP_time.fillna(1000000)
count = 0

i = 0
column = []
while i < len(HR_time.columns):
    column.append("HR_offset_%s"%i)
    i += 1
HR_time.columns = column
# print(HR)

i = 0
column = []
while i < len(SBP_time.columns):
    column.append("SBP_offset_%s"%i)
    i += 1
SBP_time.columns = column
# print(SBP)

i = 0
column = []
while i < len(DBP_time.columns):
    column.append("DBP_offset_%s"%i)
    i += 1
DBP_time.columns = column

DBP_size = DBP_res.groupby("patientunitstayid").size()
SBP_size = SBP_res.groupby("patientunitstayid").size()
HR_size = HR_res.groupby("patientunitstayid").size()

MSI = []
i = 0
while i < len(DBP.index):
    #one time deal with one patient
    goal = 0
    loop = 0
#     whether index is in DBP SBP HR
    if DBP.index[i] in HR.index.values and DBP.index[i] in SBP.index.values:
        SIpt = []
        SIpt.append(DBP.index[i])
#         take DBP columns as basic columns length check loop value under length and success under 9
        while loop < DBP_size.loc[DBP.index[i]] and goal < length_MSI:
            #whether we have every ingredient
            if DBP_time.loc[DBP.index[i]][loop] in SBP_time.loc[DBP.index[i]].values and DBP_time.loc[DBP.index[i]][loop]in HR_time.loc[DBP.index[i]].values:
                DBP_col_index = loop
                HR_col_index = int(np.where(HR_time.loc[DBP.index[i]].values == DBP_time.loc[DBP.index[i]][loop])[0][0])
                SBP_col_index = int(np.where(SBP_time.loc[DBP.index[i]].values == DBP_time.loc[DBP.index[i]][loop])[0][0])
                #caculate MSI
                msi = 0
                pulse = 0
                MAP = 0
                if (float(HR.loc[DBP.index[i]][HR_col_index]) != 0 or 1000000) and (float(SBP.loc[DBP.index[i]][SBP_col_index]) != 0 or 1000000) and (float(DBP.loc[DBP.index[i]][DBP_col_index]) != 0 or 1000000): 
                    pulse = float(SBP.loc[DBP.index[i]][SBP_col_index]) - float(DBP.loc[DBP.index[i]][DBP_col_index])
                    MAP = pulse/3 + float(DBP.loc[DBP.index[i]][DBP_col_index])
                    if MAP != 0:
                        msi = float(HR.loc[DBP.index[i]][HR_col_index]) / MAP
                        SIpt.append(msi)
                        goal += 1
            loop += 1
        MSI.append(SIpt)
    i += 1

MSI = pd.DataFrame(MSI)
column = ['patientunitstayid']
i = 0
while i < length_MSI:
    column.append("MSI_%s"%i)
    i += 1
MSI.columns = column
MSI.set_index('patientunitstayid',inplace = True)
MSI = MSI.fillna(1000000)
MSI = pd.DataFrame(MSI,dtype = np.float)

print("MSI 0 Missing Rate:",1 - MSI[MSI['MSI_0']!=1000000]['MSI_0'].count()/181244)
print("MSI 1 Missing Rate:",1 - MSI[MSI['MSI_1']!=1000000]['MSI_1'].count()/181244)
print("MSI 2 Missing Rate:",1 - MSI[MSI['MSI_2']!=1000000]['MSI_2'].count()/181244)
print("MSI 3 Missing Rate:",1 - MSI[MSI['MSI_3']!=1000000]['MSI_3'].count()/181244)
print("MSI 4 Missing Rate:",1 - MSI[MSI['MSI_4']!=1000000]['MSI_4'].count()/181244)
print("MSI 5 Missing Rate:",1 - MSI[MSI['MSI_5']!=1000000]['MSI_5'].count()/181244)
print("MSI 6 Missing Rate:",1 - MSI[MSI['MSI_6']!=1000000]['MSI_6'].count()/181244)
print("MSI 7 Missing Rate:",1 - MSI[MSI['MSI_7']!=1000000]['MSI_7'].count()/181244)
print(MSI)


MSI 0 Missing Rate: 0.3294784930811503
MSI 1 Missing Rate: 0.483254618083909
MSI 2 Missing Rate: 0.5649952550153384
MSI 3 Missing Rate: 0.6274028381629184
MSI 4 Missing Rate: 0.6790845490057602
MSI 5 Missing Rate: 0.7226059897155216
MSI 6 Missing Rate: 0.7587506345037629
MSI 7 Missing Rate: 0.7899792544856657
                      MSI_0     MSI_1           MSI_2           MSI_3  \
patientunitstayid                                                       
141168             2.100000  0.941704        1.802575        1.787234   
141203             1.028926  0.894737        0.839450        1.546392   
141227             1.243902  1.156951        1.200000        1.261803   
141229             0.798762  1.000000        1.168142        1.146667   
141284             1.268293  1.056000  1000000.000000  1000000.000000   
...                     ...       ...             ...             ...   
3353216            0.977376  0.822785        0.835443        0.798387   
3353226            0.858921  0.6

In [154]:
#show how to concat two dataframe,use for loop to connect all dataframe(must start with age_res,since its missing rate is 0)
NOAKIPT = pd.DataFrame()
NOAKIPT = pd.merge(age,nettotal,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,dialysis,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,potassium,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,sodium,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,Hct,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,BUN,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,pH,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,OI,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,TP,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,TP_sum,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,urine,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,diastolicBP,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,systolicBP,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,meanBP,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,creatinine,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,glucose,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,albumin,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,gfr,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,pulseP,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,MSI,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,weight,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,condition,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,creatinine_delta,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,urine_delta,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,diastolicBP_delta,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,systolicBP_delta,on='patientunitstayid',how='left')
# NOAKIPT = pd.merge(NOAKIPT,meanBP_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,potassium_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,sodium_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,Hct_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,BUN_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,pH_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,TP_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,glucose_delta,on='patientunitstayid',how='left')
NOAKIPT = pd.merge(NOAKIPT,albumin_delta,on='patientunitstayid',how='left')
NOAKIPT['aki'] = 0
NOAKIPT = NOAKIPT.fillna(1000000)
NOAKIPT = NOAKIPT.loc[NOAKIPT['IsDialysis'] != 1]
NOAKIPT.drop('IsDialysis',axis = 1,inplace = True)
print(NOAKIPT)
NOAKIPT.to_csv('NOAKIPT.csv', encoding='utf-8')


                    age    nettotal  potassium_0  potassium_1   sodium_0  \
patientunitstayid                                                          
141168             70.0        0.00    1000000.0    1000000.0  1000000.0   
141178             52.0  1000000.00    1000000.0    1000000.0  1000000.0   
141179             52.0  1000000.00    1000000.0    1000000.0  1000000.0   
141194             68.0  1000000.00    1000000.0    1000000.0  1000000.0   
141196             71.0  1000000.00    1000000.0    1000000.0  1000000.0   
...                 ...         ...          ...          ...        ...   
3353216            50.0    -2138.56          5.1    1000000.0      143.0   
3353226            79.0        1.41          4.8    1000000.0      136.0   
3353235            50.0        0.00          4.1    1000000.0      137.0   
3353237            79.0    -4436.50    1000000.0    1000000.0  1000000.0   
3353263            35.0       16.67          4.1    1000000.0      135.0   

           

In [49]:
HC_NOAKIPT = NOAKIPT.loc[NOAKIPT['creatinine_0'] != 1000000]
NC_NOAKIPT = NOAKIPT.loc[NOAKIPT['creatinine_0'] == 1000000]
partof_HC_NOAKIPT = HC_NOAKIPT.sample(n=14944,random_state=123,axis=0)
val_partof_HC_NOAKIPT = partof_HC_NOAKIPT.sample(n=4483,random_state=123,axis=0)
partof_HC_NOAKIPT = partof_HC_NOAKIPT.append(val_partof_HC_NOAKIPT)
partof_HC_NOAKIPT = partof_HC_NOAKIPT.drop_duplicates(keep = False)
partof_NC_NOAKIPT = NC_NOAKIPT.sample(n=2846,random_state=123,axis=0)
val_partof_NC_NOAKIPT = partof_NC_NOAKIPT.sample(n=854,random_state=123,axis=0)
partof_NC_NOAKIPT = partof_NC_NOAKIPT.append(val_partof_NC_NOAKIPT)
partof_NC_NOAKIPT = partof_NC_NOAKIPT.drop_duplicates(keep = False)
val_partof_NC_NOAKIPT = val_partof_NC_NOAKIPT.append(val_partof_HC_NOAKIPT)
partof_HC_NOAKIPT = partof_HC_NOAKIPT.append(partof_NC_NOAKIPT)

#take no aki patient id
partof_HC_NOAKIPT_index = partof_HC_NOAKIPT.values
partof_HC_NOAKIPT_index = partof_HC_NOAKIPT_index.sort()
partof_HC_NOAKIPT_index = pd.DataFrame(partof_HC_NOAKIPT_index)
partof_HC_NOAKIPT_index.to_csv('NOAKIPT_index.csv', encoding='utf-8',index = False)
'''
'''#take aki patient id
AKIPT.reset_index(inplace = True)
AKIPT_index = AKIPT.values
AKIPT_index = AKIPT_index.sort()
AKIPT_index = pd.DataFrame(AKIPT_index)
AKIPT_index.to_csv('AKIPT_index.csv', encoding='utf-8',index = False)

TrainPT = pd.DataFrame()
#partof_HC_NOAKIPT.set_index('patientunitstayid',inplace = True)
#AKIPT.set_index('patientunitstayid',inplace = True)
val_AKIPT = AKIPT.sample(n=5337,random_state=123,axis=0)
Train_AKIPT = AKIPT.append(val_AKIPT)
Train_AKIPT = Train_AKIPT.drop_duplicates(keep = False)
TrainPT = Train_AKIPT.append(partof_HC_NOAKIPT)
ValPT = val_AKIPT.append(val_partof_NC_NOAKIPT)
print(TrainPT,ValPT)
TrainPT.set_index('patientunitstayid',inplace = True)
TrainPT.to_csv('TrainPT_new_sd.csv', encoding='utf-8')
ValPT.to_csv('ValPT_new_sd.csv', encoding='utf-8')

#train
TrainPT.reset_index(inplace = True)
TrainPT_index = TrainPT['patientunitstayid']
TrainPT_index = TrainPT_index.sort_values()
TrainPT_index = pd.DataFrame(TrainPT_index)
TrainPT_index.to_csv('TrainPT_index.csv', encoding='utf-8',index = False)

#val
ValPT.reset_index(inplace = True)
ValPT_index = ValPT['patientunitstayid']
ValPT_index = ValPT_index.sort_values()
ValPT_index = pd.DataFrame(ValPT_index)
ValPT_index.to_csv('ValPT_index.csv', encoding='utf-8',index = False)

AllPT = pd.DataFrame()
AllPT = AKIPT.append(NOAKIPT)
print(AllPT)
AllPT.to_csv('AllPT_new.csv', encoding='utf-8')


/Users/rocky/opt/anaconda3/envs/py3_7/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


             BUN_0      BUN_1     BUN_delta_1      Hct_0      Hct_1  \
0             29.0  1000000.0  1000000.000000       33.4  1000000.0   
1             41.0       48.0        1.909091       28.8       25.5   
3             31.0  1000000.0  1000000.000000       35.8  1000000.0   
4             33.0  1000000.0  1000000.000000       42.8  1000000.0   
5             71.0       70.0       -0.191693       42.8       41.3   
...            ...        ...             ...        ...        ...   
1304572  1000000.0  1000000.0  1000000.000000  1000000.0  1000000.0   
2434140  1000000.0  1000000.0  1000000.000000  1000000.0  1000000.0   
2166383  1000000.0  1000000.0  1000000.000000  1000000.0  1000000.0   
1213807  1000000.0  1000000.0  1000000.000000  1000000.0  1000000.0   
1010520  1000000.0  1000000.0  1000000.000000  1000000.0  1000000.0   

           Hct_delta_1           MSI_0           MSI_1           MSI_2  \
0        1000000.00000        1.161972        1.050000        1.134021   